<a href="https://colab.research.google.com/github/Da-Pen/CS486-twitter-bot/blob/main/LSTM/CS486_LSTM_word_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from collections import defaultdict

# CONSTANTS
NEWS_ORGS_DATA_FILE_NAME = '/content/newsorgs_data'
TRUMP_DATA_FILE_NAME = '/content/donald_trump_data'
ONLY_LOWERCASE = True
SKIP_URLS = True
SKIP_ELLIPSES = True
SKIP_RETWEETS = True
SKIP_REPLIES = True     # it seems like Trump often has tweets where he simply replies to another Twitter user or quotes them. They usually start with '@' or '"@'. If this is set to true, then ignore those tweets.
MIN_TWEET_LENGTH = 50 # characters

# returns a string minus all the urls in it
def ignore_urls(s):
    return ' '.join([x for x in s.split() if 'http' not in x])


# returns True for words like 'Hello' and 'hello' but not 'HELLO' or 'HelLo'
def is_normal_capitalization(word):
    return word[1:].islower()

# replaces 'Abcdef' with 'abcdef' but leaves 'ABCDEF' and 'AbCdeF' intact
def replace_first_caps(sentence):
    return ' '.join([word.lower() if is_normal_capitalization(word) else word for word in sentence.split(' ')])

# gets a list of strings representing the tweets in the given file.
# can limit the number of tweets to get using upto.
# replaces 'NEWLINE's with actual \n characters.
def get_tweets_list(filename, upto=None):
    f = open(filename, 'r')
    lines = f.read().split('\n')[:upto]
    f.close()
    # replace NEWLINE's and ignore all lines that do not have spaces (because they are probably just a link)
    lines = [line.replace('NEWLINE', '\n') for line in lines if line.strip().find(' ') != -1]
    if ONLY_LOWERCASE:
        lines = [replace_first_caps(line) for line in lines]
    if SKIP_ELLIPSES:  # skip tweets with the '…' character, which indicates that it has been truncated
        lines = [line for line in lines if line.find('…') == -1]
    if SKIP_URLS:
        lines = [ignore_urls(line) for line in lines]
    if SKIP_RETWEETS:
        lines = [line for line in lines if line[:2] not in ('RT', 'rt')]
    if SKIP_REPLIES:
        lines = [line for line in lines if len(line) > 0 and line[0] != '@' and line[:2] != '"@']
    # # check what percentage of characters are valid: if less than MIN_VALID_CHAR_PERCENT are valid, then ignore this tweet. Otherwise, delete invalid characters.
    # lines = [filter_invalid_chars(line) for line in lines if filter_invalid_chars(line) is not None]
    return np.array(lines)

# given a list of tweets, gets a map of words to occurrences
def get_words(tweets):
    all_words = defaultdict(lambda: 0)
    for tweet in tweets:
        words = tweet.split(' ')
        for word in words:
            all_words[word] += 1
    return all_words

def get_words_list(words_map):
    min_occurrence = 5
    words_list = []
    for word in words_map.keys():
        if words_map[word] > min_occurrence:
            words_list.append(word)
    return words_list


def filter_words(tweet, words_set):
    return ' '.join([word for word in tweet.split(' ') if word in words_set])

tweets = get_tweets_list(TRUMP_DATA_FILE_NAME)
# tweets = get_tweets_list(NEWS_ORGS_DATA_FILE_NAME)

words_list = get_words_list(get_words(tweets))

# NOTE: had to save a copy of the words list for the generated models to account for a bug where for some reason different words lists would be generated different times I ran this program on different Colab runtimes

# News words list
# words_list = ['organizers', 'announced', 'that', 'canadian', 'singer', 'the', 'will', 'perform', 'at', '2021', 'super', 'show.', 'new', 'projections', 'by', "ontario's", 'science', 'advisory', 'table', 'show', 'pandemic', 'is', 'worsening', 'across', 'province', 'quebec', 'premier', 'françois', 'legault', 'says', 'government', 'must', 'consider', 'all', 'options', 'to', 'curb', 'spread', 'of', 'COVID-19,', 'including', 'winter', 'holiday', 'break', 'in', 'international', 'force', 'peace', 'agreement', 'said', 'eight', 'were', 'killed', 'when', 'one', 'its', 'crashed', 'during', 'a', 'mission', 'number', 'COVID-19', 'cases', 'per', 'day', 'on', 'rise', 'states,', 'and', 'deaths', 'from', 'federal', 'has', 'plans', 'help', 'more', 'people', 'living', 'hong', 'kong', 'come', 'canada', 'as', 'chinese', 'down', 'pro-democracy', 'movement', 'may', 'have', 'broken', 'law', 'protests', 'RCMP', '@cattunneycbc', 'A', 'nova', 'scotia', 'man', 'launched', 'lawsuit', 'against', 'organization', 'runs', 'where', 'he', 'others', 'abused', 'for', 'years.', 'knew', 'it', 'was', 'wrong', 'her', 'sister', 'with', 'yet', 'she', 'went', 'great', 'cover', 'up', 'this', 'information.', 'canadians', 'deserve', 'better', 'their', 'elected', '—', 'conservative', 'critic', 'michael', '@AshleyBurkeCBC', 'U.S.', 'food', 'drug', 'administration', 'approved', 'promising', 'antibody', 'therapy', 'scientists', 'treat', 'mild', 'moderate', 'coronavirus', 'infections.', '@JPTasker', '"There', 'no', 'time.', 'we', 'act', 'now.', 'need', 'something', 'strong', 'mandatory', 'order', 'dr.', 'windsor', 'regional', 'hospital', 'CEO', 'david', 'drove', 'pick', 'an', 'patient', "didn't", 'enough', 'stock', 'available', 'typhoon', 'areas', 'passed', 'over', 'two', 'million', 'are', 'without', 'power', 'metro', 'surrounding', 'area.', 'told', 'pay', 'balance', 'after', 'speaking', 'someone', 'virgin', 'mobile', 'about', 'expensive', 'president-elect', 'joe', 'biden', 'appointed', 'top', 'democratic', 'official', 'ron', 'klain', 'his', 'chief', 'staff.', 'charge', 'obama', "administration's", 'response', 'health', 'crisis', 'independent', 'And', "they're", 'advocates', 'say', 'step', 'efforts', 'fleeing', 'conflict', 'abroad', 'despite', 'lockdowns', 'travel', 'restrictions.', '@OttawaReporter', 'opposition', 'final', 'before', 'protest', 'four', 'every', 'remembrance', 'what', 'soldiers', 'some', 'already', 'project', 'ontario', 'surpass', '2,000', 'early', '@CBCQueensPark', 'james', 'song', 'keep', 'point', 'view', 'would', 'fears', 'transmission', 'around', 'st.', 'really', 'set', 'into', 'motion', 'first', 'also', 'leave', 'position', 'legal', 'analyst', 'CNN.', "province's", 'newest', 'public', 'sets', 'indoor', 'outdoor', 'size', 'five', 'people,', 'agency', 'own', 'authority', 'not', 'nations', 'do', 'things,', 'expert', 'says.', 'restrictions', 'could', 'include', 'things', 'like', 'capacity', 'services', 'activities', 'among', 'them', 'nearly', '200', 'queens', 'had', 'potential', 'start', 'there', 'few', 'but', 'still', 'development', 'rapid', 'testing', 'vaccines', 'novel', 'ensure', 'summer', 'games', 'can', 'successfully', 'take', 'place', 'tokyo', 'next', 'year,', 'starting', 'july', 'toward', 'dropped', '29', 'cent', 'pre-pandemic', '21', 'border', 'get', 'discovery', 'conservation', 'asked', 'court', 'father', 'return', 'role', 'global', 'management', 'agreed', 'share', 'company', 'price', 'very', 'good', "here's", 'latest', 'how', 'rest', 'world.', 'success', 'keeping', 'rates', 'low', 'change', 'coming', 'weeks,', 'according', '@TO_jwo', 'newly', 'confirmed', 'infections', 'push', 'seven-day', 'average', 'daily', 'highest', 'been', 'any', 'pandemic.', 'former', 'parks', 'manager', 'anonymous', 'restaurants', 'grocery', 'private', 'being', 'planned', 'long-term', 'care', 'facilities', 'home', "canada's", 'oldest', 'many', 'whom', 'particularly', 'high', 'risk', 'COVID-19.', 'centre', 'be', 'spot', 'police', 'community', 'groups', 'drop', 'off', 'person', 'overnight', 'instead', 'sending', '15', 'lawmakers', 'following', 'move', 'fellow', 'storms', 'longer', 'ocean', 'expert.', '@NebulousNikki', 'death', 'toll', 'again', 'begun', 'than', '600', 'since', 'late', 'august,', 'half', 'which', 'occurred', 'just', 'three', 'weeks.', 'who', 'should', 'vaccine', 'extremely', 'challenging', 'data', '@adamsmiller', '"It', 'so', 'said,', 'through', 'face', 'LIVE', '|', 'CBC', 'news', 'special', 'made', 'covered', 'took', '300', 'hours', 'our', 'annual', 'going', 'virtual', 'way', 'make', 'experience', 'future', '@Murray_Brewster', 'protests.', 'now', 'they', 'both', 'veterans', 'second', 'world', 'war.', 'fox', 'remarks', 'those', 'trump.', 'chose', 'toronto', 'because', 'demand', 'seeing', 'users', 'country,', 'executive', 'scott', "tuesday's", 'numbers', 'continue', 'phase', 'largely', 'lower', 'strict', 'guidelines', 'place.', 'hundreds', 'died', 'air', 'fighting', 'escalating', 'fear', 'civil', 'war', 'given', 'deep', 'between', 'prime', 'minister', 'abiy', 'comes', 'largest', 'ethnic', 'provides', 'opportunity', 'used', 'same', 'or', 'measures', 'under', 'section', '22', 'city', 'require', 'meeting', 'event', 'halls', 'other', 'remain', 'massive', 'explosion', 'port', 'district', 'buildings', 'within', 'killing', 'justices', 'heard', 'arguments', 'appeal', 'coalition', '20', 'states', 'california', 'york', 'house', 'representatives', 'hoping', 'preserve', 'doctors', 'health-care', 'workers', 'warning', 'hospitals', 'able', 'handle', 'rising', 'cases.', 'staffers', 'employees', 'them,', 'appearances', 'publicly', 'work.', 'amazon', 'fines', 'billion', 'US', 'faces', 'antitrust', 'results', 'showing', 'won', 'election,', 'dismissed', 'questions', 'whether', 'lost', 'judge', 'election', 'president', "trump's", 'claims', 'fraud', 'polls.', 'statement', 'officials', 'reporting', 'called', 'country.', '"We', 'truly', 'fight', 'officer', 'brent', 'tuesday', 'morning', 'announcement', 'brian', 'total', 'includes', 'consecutive', 'well', 'peel', 'region,', '100', 'region', '50', 'NDP', 'leader', 'thinks', 'basic', 'income', 'pilot', 'newfoundland', 'labrador', 'fuel', 'national', 'program.', "it's", '1', 'trial.', "he's", 'last', 'him', 'months', 'clear', 'name.', 'alberta', 'calling', 'two-week', 'lockdown', 'homes', 'repeatedly', 'violated', 'provincial', 'rules.', 'martin', "country's", 'congress', 'corruption', 'update:', 'trudeau,', 'address', 'defence', 'staff', 'general', 'asking', 'join', 'campaign', 'award', 'cross', 'soldier', 'you', 'university', 'professor', '$1', 'gold', 'work', '@mle_chung', '#Analysis', 'committed', 'climate', 'change.', 'pressure', 'more.', '73', 'years,', 'john', 'visited', 'grave', 'fallen', 'immigration', 'lawyer', 'pandemic,', 'detention', 'don', 'mortgage', 'bad', 'interest', '2020', 'prize', 'awarded', 'collection', 'remove', 'parts', 'voters', 'chance', 'polls', '9', 'a.m.', '8', 'p.m.', 'friday.', 'monday,', 'support', 'known', 'bill', 'believed', 'always', 'represents', 'regardless', 'why', 'wear', 'it.', 'believe', 'conversation', 'choice', 'signed', '70', 'control', 'armenian', 'forces', 'backed', 'armenia', 'ended', 'estimated', '30,000', 'died.', 'deal', 'monday', 'sell', 'partnership', 'british', 'columbia', 'briefing', 'weekend', 'sweeping', 'vancouver', 'coastal', 'regions', 'period', 'surge.', 'list', 'eligible', 'people.', 'recall', 'certain', '2018', 'expected', 'storm', 'likely', 'marks', 'beginning', 'trump', 'conceded', 'claiming', 'evidence', 'conspiracy', 'democrats', 'vote', 'tally', "biden's", 'analysis', 'far', 'looked', 'seven', 'days', 'pfizer', 'look', '14', 'adds', 'uncertainty', 'transition', 'nov.', '3', 'vote.', 'refused', 'concede', "week's", 'biden,', 'jan.', 'association', 'nothing', 'lawyers', 'making', 'financial', 'various', 'political', 'donations', 'test', 'positivity', 'rate', 'manitoba', 'rose', 'these', 'men', 'reported', 'alleged', 'He', 'justin', 'trudeau', 'spoke', 'today', 'shared', 'battle', 'attorney', 'criminal', 'investigation', 'donald', 'defamation', 'women', 'sexual', 'liberal', 'spend', 'dollars', 'connect', 'most', 'high-speed', 'internet', 'approval', 'process', 'confidence', 'safe', 'effective.', 'markets', 'result', 'elections,', 'saw', 'democrat', 'win', 'presidency.', "pfizer's", 'trial', 'secured', 'millions', 'said.', 'filed', 'alleging', 'job', 'allow', 'widespread', 'use', 'hyperloop', 'completed', "world's", 'passenger', 'ride', 'system,', 'executives', 'hit', 'las', 'kids', 'phone', 'handling', 'increased', 'even', 'difficult', 'demands', 'tropical', 'eta', 'landfall', 'key', 'sunday', 'night,', 'leaving', 'scores', 'dead', 'missing', 'mexico', 'central', 'america.', '#Analysis:', 'radio', 'host', 'topped', 'supporters', 'skeptical', 'progressive', 'policies', 'economy', '@markgollom', 'airline', 'industry', 'contingent', 'providing', 'passengers', 'whose', 'flights', 'cancelled', 'suggests', 'shots', '90', 'effective', 'preventing', "doesn't", 'mean', 'woman', 'officers', 'prison', 'heading', 'supreme', 'end', 'practice', 'halifax', 'students', 'raised', 'thousands', 'victims', 'mass', 'shooting', 'while', 'walking', 'route', 'aides', 'allies', 'acknowledged', 'privately', 'best', 'outbreak', 'infected', '11', 'residents', 'report', 'cases,', 'family', 'members', 'out', 'B.C.', 'focus', 'social', 'group', 'exercise', 'decisions', 'united', 'recorded', 'million,', 'seen', 'ont.', 'rallied', 'weekend.', 'came', 'observers', 'see', 'prince', 'william', 'changes', 'him.', "georgia's", 'so-called', 'state,', 'reliable', 'republican', 'presidential', 'wake', 'real', 'estate', 'experts', 'search', 'space.', 'foreign', 'affairs', 'advance', 'theresa', 'tam', 'several', 'accelerated', 'schools', 'spent', 'trying', 'sort', 'quality', 'issues.', 'prove', 'another', 'challenge.', 'hockey', 'night', 'icon', 'NHL', 'star,', 'age', 'falling', 'short', 'diversity', 'numbers.', 'TV', 'alex', 'trebek', 'long', 'career', 'back', 'years', 'upcoming', 
#  'ban', 'single-use', 'celebrations', 'put', 'plans.', 'BREAKING:', '80', 'edmonton', 'heavy', 'saturday.', 'couple', 'left', 'brunswick', 'gained', 'media', 'never', 'go', 'away.', 'mark', 'nation', '@Alex_Panetta', 'shows', 'viewers', 'tree', 'hot', 'days,', 'tweet', 'led', 'actor', 'arctic', 'space', 'station', '40', 'ago,', 'ken', 'record,', 'named', 'red', 'sunday,', 'sun', 'showed', 'up.', 'sure', 'exactly', 'happened', 'ont.,', 'launching', 'reading', 'service', 'proposed', 'drivers', 'downtown', 'critics', 'hurt', 'tiny', 'alaska,', 'attend', 'school', 'actually', 'getting', 'does', 'winnipeg', 'warn', 'curfew', 'amount', 'play', 'love', 'received', 'island', 'until', 'budget', 'sign', 'doug', 'ford', 'addresses', 'americans', 'time', 'star', 'paul', 'lee', 'emotional', 'moment', 'once', 'nation,', 'cities', 'worst', 'infection', 'canada.', "can't", "won't", 'become', 'again.', '@AaronWherry', 'quickly', 'turn', 'surge', 'least', 'prevent', 'worse.', 'looks', 'forward', 'building', 'relationship', 'biden.', 'crossed', '270', 'electoral', 'college', 'votes', 'pennsylvania.', 'documents', 'obtained', 'news,', 'patients', 'intensive', 'units', 'friday', 'june.', 'america', 'wins', 'election.', 'state', 'results.', 'politics,', 'energy', 'firms', 'jobs', 'struggling', '@KyleBakx', '@TonySeskus', 'mail-in', 'ballots', 'slim', 'candidates', 'delay', 'voter', 'here', 'answers', 'your', 'questions.', 'registered', 'voters,', 'though', 'turnout', 'recent', "there's", 'turned', 'beat', 'friday,', '28', 'care.', 'sitting', 'room', 'watching', 'movie', 'wearing', 'masks', 'usual', 'protocols', 'series', 'turner', 'pulled', 'game', 'tested', 'positive', "There\'s", 'generation', 'along', 'maybe', "don't", 'idea', 'lot', 'if', 'japanese', 'discovered', 'middle', 'record', 'little', 'bit', 'chaotic', 'COVID', 
#  'elections.', 'changing', 'market', 'recovery', 'october,', 'only', 'jobs.', 'added', 'smallest', 'began', 'month.', 'FBI', 'virginia', 'philadelphia', 'police.', 'timeline', 'warns', 'doses', 'first.', 'presidency', 'might', 'relations', 'russia', 'appeared', 'propaganda', 'based', 'whole', 'foods', 'poppies', "weren't", 'allowed', 'recently', 'updated', 'locations', "wouldn't", 'rejected', 'requests', 'minneapolis', 'charged', 'george', 'ordered', 'tried', 'together', 'airlines', 'schedule', 'tens', 'monthly', 'cancel', 'vast', 'majority', 'weeks', "It's", 'strategy', 'free', 'cannabis', 'concerned', 'stop', 'using', 'opioid', 'treatment.', 'places', 'enormous', 'support.', 'fact', "pennsylvania's", 'county', '2016', 'explain', 'born', 'leads', 'pennsylvania', 'conference', 'important', 'unfold', "he'll", 'illegal', 'businesses', 'The', 'claimed', 'responsible', 'losses', 'suffered', '2008', 'outbreak.', '#Opinion:', 'judges', 'neither', 'nor', 'ultimately', 'saying', 'is,', 'be.', 'figure', 'means', 'fewer', 'paid', 'did', 'animals', 'change,', 'MPs', 'growing', 'online', 'security', 'threats', 'directed', 'public.', 'wait', 'numbers,', 'suggested', 'burden', 'cannot', '@don_pittis', 'month', 'started', 'candidate', 'ground', 'battleground', 'georgia', 'parliament', 'bring', 'existing', 'suicide', 'prevention', 'looking', 'ways', 'meet', "aren't", 'broke', 'season', 'blue', 'detailed', 'case', 'due', 'technical', 'problems', 'unclear', 'letter', 'door', '"The', 'situation', 'needs', 'bonnie', 'henry', 'active', 'risen', 'multiple', 'studies', 'shown', 'light', 'found', 'streets', 'affect', 'august', 'fall', 'season,', 'production', 'season.', 'elections', 'spokesperson', 'having', 'nonpartisan', 'politics', 'voting', 'process.', 'small', 'amid', 'wave', 'tax', 'announce', 'direct', 'funding', 'billions', 'aid', 'helped', 'replace', 'costs', 'companies', 'facing', 'revenue', 'limit', 'puts', 'strain', 'areas.', 'officially', 'paris', 'countries', 'increases', 'below', 'century.', 'delayed', 'almost', 'handful', 'maintains', 'lead', 'experts,', 'intervene', 'stage', 'seemed', 'seem', 'know', 'counted.', 'germany', '20,000', 'day,', 'level', 'yet.', 'israel', 'village', 'west', 'such', '"I', 'think', 'meant', 'arm', 'GM', 'pickup', 'assembly', 'plant', 'labour', 'union', '@p_evans', 'town', 'bracing', 'rally', 'england', 'entering', 'venues', 'stores', 'selling', 'round', 'save', 'life.', 'crusade', 'cut', 'spending', 'biggest', 'deficit', 'history.', 'all-time', 'day.', 'major', '100,000', 'store', 'call', 'recommended', 'filter', 'mask,', 'bought', 'shopping', 'telescope', 'B.C.,', 'astronomers', 'powerful', 'fast', 'presence', 'team', 'philadelphia,', 'dozens', 'protesters', 'ready', 'down,', 'tensions', 'counting', 'votes.', 'engaged', 'recount', 'wisconsin', 'filing', 'lawsuits', 'pennsylvania,', 'michigan', 'georgia.', 'economists', 'worry', 'divide', 'dire', 'economic', 'follow', '2018,', 'canada,', '13', 'lives', 'coal', 'taken', 'line.', 'vote-counting', 'centers', 'detroit', 'phoenix', 'returns', 'states.', 'moved', 'closer', 'victory', 'race', 'wisconsin.', 'danish', 'virus', 'became', '500', 'jones', 'afternoon', 'fared', 'better.', 'begins', 'count', 'workplace', 'declared', 'linked', 'makes', 'us', 'accurate', 'picture', 'reached', 'parties', 'provide', 'rent', 'relief', 'fashion', 'bid', 'over.', 'posted', 'himself', 'twitter', 'front', 'map', 'term', "we're", 'want', 'falsely', 'apply', 'week.', 'extra', 'required', 'previously', 'flow', 'goods', 'statistics', 'ahead.', 'rules', 'ballots,', 'causing', 'delays', 'access', 'numerous', 'opening', 'ballots.', 'anyone', 'less', 'popular', 'information', 'mayor', 'threatened', 'divided', 'masks.', 'chair', 'task', 'efficacy', 'safety', 'advanced', 'investigating', 'rare', 'flu', 'detected', 'There', '27', 'influenza', 'concern', 'threshold', 'too', 'single', '"murder', 'advised', 'hurricane', 'continues', 'coast', 'much', 'tuesday,', 'setting', 'deadly', 'landslides', 'ballot', 'proposal', 'uber', 'status', 'rather', 'employees.', '7', 'remained', 'morning.', 'carbon', 'emissions', 'car', 'earth', 'times.', 'formally', 'promise', 'six', 'states:', 'nevada,', 'georgia,', 'arizona,', 'michigan,', '@EricGrenierCBC', 'miss', 'possible', 'glass', 'oregon', 'amounts', 'street', 'drugs', 'huawei', 'suing', 'agencies', 'try', 'release', 'believes', 'reveal', 'behind', 'arrest', 'post-election', 'october', 'resort', 'buy', 'created', 'lottery', 'them.', 'cold', 'weather', 'mental', '5', 'prepare', 'reports', 'close', 'expressed', 'tight', 'ongoing', 'counts', 'swing', 'shares', 'trading', 'hands', 'opened', 'above', 'offers', 'investors', 'offer', 'served', 'flight', 'royal', 'american', 'continued', 'wednesday', 'winner', 'declaration', 'race.', 'joining', 'coverage', 'find', 'here:', 'prospects', 'stimulus', 'recover', 'americans.', 'wisconsin,', 'coast,', 'races', 'contested', 'north', 'carolina,', '"This', 'large', 'challenge', 'poll', 'surveyed', 'system', 'marked', 'hand', 'maintaining', 'physical', 'distance', 'part', 'sustained', 'urging', 'stay', 'home,', 'live', 'appear', 'site', 'in,', 'president,', 'senate', 'house.', 'winds', 'kilometres', 'center.', 'U.S.,', 'recommendations', 'officials.', 'history,', 'ottawa', 'policy', 'streaming', 'platforms', 'faced', 'traditional', 'jason', 'listen', 'advice', 'main', 'recommending', 'choose', 'intended', 'clarity', 'level.', 'gain', 'senate,', 'flip', 'depending', 'presidency,', 'deciding', 'monitoring', 'urged', 'additional', 'single-day', 'province.', 'regulators', 'critical', 'reaching', 'virus.', 'teams', 'option', 'considered', 'erin', "O'Toole", 'conservatives', 'simply', 'open', 'lose', 'training', 'bias', 'internal', 'showdown', '2000', 'lessons', "today's", 'life', 'construction', 'raise', 'build', 'down.', 'She', 'months,', 'away', 'young', 'girl', 'rescued', 'rubble', 'collapsed', 'izmir,', 'earthquake', 'seasonal', 'type', 'depression', 'brought', 'winter,', 'that’s', 
#  'produced', 'unusual', 'moments', 'windows', 'white', 'children', 'got', 'haul', 'initial', 'eastern', 'long,', 'dark', 'winter.', 'rainfall', 'predicted', 'region.', 'awards', 'book', 'human', 'rights', 'area', 'targeted', 'gives', 'shut', 'block', 'road', 'near', 'rural', 'bodies', 'fourth', 'injured', 'secure', '2016,', 'moving', 'wife', 'kelly', 'peaceful', 'loses', 'that,', 'trump,', 'anything', 'tend', 'considering', 'survive', 'member', 'household', 'symptoms', 'entire', 'transport', 'detect', 'endangered', 'atlantic', 'right', 'interior', 'vienna', 'terror', 'attack.', "B.C.'s", 'historic', 'preparing', 'plan', 'giving', 'bloc', 'blanchet', 'doubling', 'draw', 'line', "party's", 'values', 'kept', 'diagnosis', 'secret', 'alarm', 'reported.', 'waiting', 'hosted', 'fair', 'attended', 'iranian', 'spokesman', 'involved', 'voluntarily', 'taking', 'review', 'rideau', 'culture.', 'readers', 'send', 'protect', 'full', 'patients.', 'remains', 'chances', 'hillary', 'ago.', 'Donald', 'pull', 'ruled', 'owner', 'published', 'rescue', 'girls', 'alive', 'apartment', 'turkish', '21,', 'stand', 'charges', 
#  'assault', '2017', 'alcohol', 'system.', 'note', 'prepared', 'earlier', 'year', 'deputy', 'greater', 'intelligence', 'here,', 'gathered', 'family,', 'friends', 'colleagues', 'formula', 'driver', 'lewis', 'hamilton', 'grand', 'different', 'nine', 'finally', 'tell', 'coronavirus,', 'story.', 'consumer', 'germany,', 'avoid', 'travel.', 'crown', 'washington', "isn't", 'impact', 'processes', 'determine', 'disease', 'equivalent', 'category', 'boris', 'johnson', 'current', 'threaten', 'laid', 'children.', 'outcome', 'networks', 'soon', 'call.', 'local', 'cult', 'journalists', 'past', '24', 'began.', 'finance', 'trail', 'territory', 'permanent', "nation's", 'bringing', "city's", 'intense', 'capital', 'appears', 'arrived', 'later,', '2020.', 'finds.', 'arrested', 'male', 'suspect', 'injured,', 'sent', 'crashing', 'province,', 'hospitalized', 'fire', 'chris', 'decided', '2018.', 'gatherings', 'learned', 'bank', 'governor', 'reducing', 'medical', 'business', 'owners', "they've", 'shot', 'military', 'essential', 'close,', 'immune', 'board', 'receive', 'regular', 'ends', 'hour', 'sleep', 'worth', 'it?', 'economy.', 'prospect', 'big', 'halloween', 'watch', 'movies', 'it,', 'run', 'mind', 'events', 'distancing,', 'florida.', 'antibodies', 'sean', 
#  'bond', 'threatening', 'bob', 'money', 'mississippi', "she's", 'politically', '2019.', '1,', '@cbcasithappens', 'felt', '"the', 'recovered', 'earlier.', 'changed', 'bay', "hasn't", 'suggest', 'golf', 'played', 'struggled', 'problem', 'worse', 'northern', 'P.E.I.', "man's", 'website', 'allows', 'visitors', 'read', '@trevorjdunn', 'mother', 'equipment', 'necessary', 'stronger', 'anxiety', 'looming', 'montreal', 
#  'scrambling', 'warming', 'spaces', 'beds', 'airport', 'program', '12', 'projects', 'party', 'emergency', 'aim', 'It\'s', 'politics.', "that's", 'kind', 'nature', 'turning', 'updates', 'alert', 'app', 'exposure', 'date', 'actress', 'sentence', 'authorities', "alberta's", 'health,', 'outbreaks', 'ahead', 'older', 'brother', 'released', 'third', 'plastic', 'identified', 'citizen', 'spreading', '@ybrend', 'thought', '17', 'sound', 'protected', 
#  'immigrants', 'jack', 'black', 'ad', 'aimed', 'turkey,', 'aegean', 'prompted', 'write', 'action', 'coronavirus.', 'commons', 'gave', 'easier', 'seek', 'london,', 'personal', 'worker', 'failed', 'pet', 'investigators', 'attack', 'french', 'church', 'grew', 'domestic', 'But', 'overall', 'activity', 'february', 'sea', 'turkey', 'western', 'pushing', 'brink', '1,000', 'saskatchewan', 'finished', 'victories', 
#  'basis', 'false', 'contained', 'trend', 'towards', 'haven’t', 'up."', 'narrow', 'my', 'I', 'speak', 'language', 'expects', 'increase', '2021.', 'immediate', 'taylor,', 'army', 'peter', 'massachusetts',
#   "year's", 'director', 'cameron', 'resident', 'law,', 'convicted', 'crime', 'holds', '10', 'serving', 'world,', 'guns', 'upon', 'ruling', 'justice', 'reality', 'allegations', 'abuse', "state's", 'continuing', 'calls', 'medicine', 'prices', 'cancer', 'account', 'granted', 'request', 'privacy', 'mail', 'sunday.', 'population', 'researchers', 'study', 'party,', 'senators', 're-elected', '3.', 'quarter,', 'previous', 'one.', 'european', 'leaders', 'faith', 'digital', 'collected', 'images', 'technology', 'knowledge', 'commissioner', 'probe', 'teachers', 'stress', 'related', 'finding', '@jonmontpetit', 'committee', 'stuck', 'WE', 'charity', 'coral', 'reef', 'barrier', 'healthy', '120', 'thursday,', 'france', 'fresh', 'path', 'growth', 'operations', 'giant', 'quarantine', 'zeta', 'louisiana', 'argues', 'mix', 'competing', 'shifts', 'wreak', 'mountain', 'battles', 'then', 'nurses', 'influx', 'resurgence', 'hospitals.', 'armed', 'knife', 'nice,', 'It', 'dog', 'week,', 'projected', 'night.', 'reopening', 'caught', 'rock', 'popularity', 'TikTok.', 'responded', 'complaints', 'masks,', 'issues', 'heavily', 'longtime', '25', 'water', 'northwestern', 'clean', '30', 'stranded', 'friend', 'miles', 'taylor', 'working', 'department', 'combat', 'misconduct', 'gripped', 'described', 'relatives', 'suffering', '2010', 'body', 'episode', 'closing', 'bars,', 'sharp', 'governments', 'sought', 'fill', 'demanding', 'reporter', 'south', 'failure', 'gets', 'there.', 'ability', 'underway', 'ads', 'widely', 'media.', 'answering', 'This', 'time,', 'testing.', 'offering', 'controversial', 'pages', 'cast', 'terms', 'takes', 'declare', 'milestone', '10,000', 'That', 'true', 'ties', 'platform', 'dozen', 'senior', 'me', 'voters.', 'power.', 'trouble', 'president.', 'designed', 'slow', 'throughout', 'claim', 'superior', 'court,', 'oxford', 'retail', 'duty', 'natural', 'gas', 'reviewed', "quebec's", 'environmental', 'worried', 'meng', "ottawa's", 'reduce', 'community.', 'effort', 'species', 'artists', "president's", 'collins,', 'tom', 'office', 'doctor', 'author', 'shift', 'physician', 'delivered', 'writing', 'hearing', 'court.', 'hotel', 'bush', 'kill', '"an', 'waste', 'professional', 'raniere,', 'sex', 'keith', 'sentencing', 'female', 'followers', 'infectious', 'homeless', 'inspired', 'research', 'london', 'protection', 'southern', 'commission', 'sparked', 'uptick', 'cuts', 'adding', 'alta.,', 'hope', 'negative', 'consequences', 'right-wing', 'version', 'appearing', 'e-commerce', 'TikTok', 'products', 'app.', 'hearings', 'crimes', "government's", 'eyes', 'signs', 'bomb', 'blast', 'islamic', 'message', 'ignore', 'mandate', 'spring,', 'treatments', 'odds', 'further', 'beyond', 'think,', 'stretched', 'anywhere', 'florida', 'say.', 'reason', 'levels', 'disputed', 'brings', 'vacant', 'resigned', 'august.', 'sports', 'hall', 'fame', 'closures', 'lifted', 'thursday.', 'andrew', 'studio', 'depend', 'naval', 'oil', 'english', 'asian', 'hornet', 'nest', 'surface', 'ice', 'NASA', 'held', 'today.', 'head', 'steady', 'climb', 'mask', 'post', 'requested', 'devices', "NASA's", 'OSIRIS-REx', 'spacecraft', 'asteroid', 'particles', 'determined', 'washington,', 'question', 'accurately', 'azerbaijan', 'accusing', 'each', 'violating', 'Nagorno-Karabakh.', 'mike', 'pence', 'session', 'demanded', 'transportation', 'it’s', 'wind', 'year.', 'record-breaking', 'campaigning', 'combined', 'nationwide', 'hopes', 'battery', 'display', 'entirely', 'putting', 'funeral', 'comfort', 'restore', 'park', 'now,', 'aims', 'battered', 'campaign.', 'none', 'boost', 'data.', 'princess', 'raising', 'concerns', 'admitted', 'million.', '@sophiaharrisCBC', 'spike', 'straight', 'software', 'warned', 'reasons', 'campaigns', 'student', 'chess', 'player', 'wireless', 'entrepreneur', 'guard', 'imposed', 'closure', 'compared', 'hard', 'suit', 'ask', 'veteran', 'poland', 'influence', 'acting', 'priority', 'commercial', 'sector', 'accept', 'diagnosed', 'one-day', 'seeking', 're-election', 'later', 'challenges', 'secretary', 'democracy', 'integrity', 'snap', 'constitution', 'act.', 'saturday,', '@bethanylindsay', 'programs.', 'skin', 'risks', 'parents', 'children,', 'begin', 'january', 'contract', 'aware', 'expanding', 'disease,', 'track', 'currently', 'video', 'hollywood', 'remaining', '&amp;', 'same.', 'demonstrations', 'unit', 'explore', 'severe', 'forecast', 'arrive', 'environment', 'celebrating', 
#  'council', 'festival', 'separated', 'border.', 'famous', 'sold', 'US,', 'ever', 'bird', 'hate', 'suspected', 'unknown', 'election?', 'inform', 'coverage.', 'email', 'legislation', 'harder', 'country', 'toronto,', '@LaurenPelley', 'thursday', 'evening', 'imagine', 'point.', 'U.K.', 'EU', 'negotiations', 'trade', 'lays', '“the', 'greatest', 'lead.', 'identify', 'workers,', 'brief', 'affordable', 'housing', 'especially', 'serious', 'reveals', 'repeat', 'marched', 'carrying', 'withstand', 'animal', 'studying', 'planes', 'buildings.', 'willing', "island's", 'So', 'attempts', 'holding', 'advantages', 'cash', 'and,', 'hiring', 'staff,', 'creating', 'plan,', "china's", 'definition', 'unveiled', 'spring', 'works', 'offered', 'land', 'calgary', 'picked', 'speed', '14th', 'century', 'merely', 'temperatures', 'days.', 'debate', 'carried', 'ET.', 'itself', 'tim', 'burger', 'king', 'customers', 'minnesota', 'murder', 'second-degree', 'judiciary', 'voted', 'favour', 'nomination', 'amy', 'coney', 'barrett', 'learn', 'residential', 'education', 'korean', 'gunman', 'astronaut', 'provinces', 'artist', 'fierce', 'winning', 'barack', 'views', 'philadelphia.', 'rudy', 'giuliani', 'featured', 'borat', 'scene', 'demonstrating', 'cities,', 'feared', 'accused', 'senate.', 'race,', 'helping', 'effects', 'contain', 'june', 'progress', 'registration', 'caused', 'opposed', 'barrett,', 'appeals', 'confirmation', 'expand', 'judicial', 'records', 'wednesday,', '@sarahcrgr', 'iran', 'met', 'delegation', 'week', 'talks', 'figures', 'michigan.', 'postal', 'employees,', 'vehicles', 'appearance', 'tonight', 'month,', 'restrictions,', 'criticism', 'approach.', 'effectively', 'paying', 'stopped', 'purdue', 'wealthy', 'advisers', 'chosen', '4', 'give', 'preliminary', 'september', 'sales', 'running', 'ending', 'double', 'halt', 'pope', 'francis', 'endorsed', 'same-sex', 'unions', 'corporate', 'individuals', 'liberals', 'found.', 'representing', 'families', 'google', 'moves', 'tech', 'china', 'reach', 'disinformation', 'measure', 'backing', 'resources', 'assistant', 'olympics', 'significant', 'brutality', 'japan', 'bill.', 'steadily', 'fake', 'encouraging', 'busy', 'freeze', 'nuclear', 'extend', 'arms', 'treaty', 'jeff', 'bridges', 'voice', 'character', '"As', 'sept.', 'launch', 'lobster', 'sued', 'advertising', 'competition', 'seniors', 'staying', 'safe,', 'nursing', 'included', 'turns', 'out,', 'professors', 'defending', 'colleague', 'suspended', 'class', 'benefits', 'youth', 'programs', 'playing', 'appointment', 'infections,', 'useful', 'probably', 'hardest', 'decision', 'pumpkin', 'condition', 'statue', 'times', 'larger', 'summer.', 'approach', 'advocacy', 'wants', 'conversations', 'vulnerable', 'shutting', 'effect', 'damage', 'degrees', 'music', 'usually', 'hotels', 'rates,', 'classes', 'home.', 'dealing', 'anthony', 'fauci', 'agree', 'mostly', 'uncertainty,', 'survey', 'november,', 'russian', 'attacks', 'wide', 'range', 'champagne', 'wall', 'migrants', 'narrowly', 'funded', 'managed', 'gary', 'league', 'fans', 'closed', 'countries.', 'original', 'selected', 'network', 'moon,', 'humans', "john's", 'indigenous', 'impose', 'apart', 'messages', 'telling', 'stories', 'gaining', 'strength', 'consumers', 'auto', 'attacked', 'violent', 'crucial', 'racism', 'shows.', 'blocking', 'thai', 'What', 'fire.', 'widow', 'locked', 'apple', 'cabinet', 'ministers', 'joined', 'hold', 'discuss', 'southwest', 'native', 'perfect', 'counted', 'childhood', 'starts', 'engaging', 'wounded', 'UN', '"a', 'name', 'monday.', 'not.', 'contact', 'tracing', 'depends', 'texas', 'physically', 'distancing', 'in-person', 'attention', 'Tuesday.', 'sample', 'reopen', 'oct.', 'however,', 'scheduled', 'story', 'two-thirds', 'laura', 'ones', 'inside', 'treatment', 'not,', 'normal', 'school.', 'done', 'standard', 'edward', 'county.', 'complete', 'wildlife', 'estimate', '2015', 'volunteers', 'expect', 'doing', 'decades', 'socially', 'ever.', 'on,', '60', 'zealand', 'jacinda', 'office.', 'polling', "wasn't", 'steps', 'happening', 'delivery', 'orders', 'govern', 'operation', 'fully', 'canada’s', 'hear', 'leaders,', 'systemic', 'healthcare', 'developed', 'school,', 'monitor', 'san', 'lucky', 'opportunities', 'emails', 'tied', 'lived', 'son', 'fund', 'produce', 'involving', 'conducted', 'daughter', 'decades.', 'patrol', 'retired', 'argued', 'expecting', 'flying', 'fossil', 'heat', '700', 'fifth', 'violence', 'harassment', 'sen.', 'ben', 'christian', 'clinical', 'visit', 'dying', 'visits', 'syndrome', 'boards', 'model', 'remote', 'leading', 'doesn’t', 'drive', 'benefit', 'measures,', 'wondering', 'control.', 'maps', 'names', 'industry.', 'firm', 'market.', 'tony', 'jury', 'refuses', 'gov.', 'gretchen', 'whitmer', 'plot', 'tonight,', 'debate,', 'temporarily', 'restricted', 'three-day', 'As', 'today,', 'country’s', 'field', 'milwaukee', 'suburb', "barrett's", 'nomination.', 'grant', 'asylum', 'criticized', 'jerry', 'dispute', 'virtually', 'queen', 'elizabeth', 'associated', 'kamala', "harris's", 'vice-presidential', 'refusing', 'customer', 'restaurant', 'september,', 'electric', 'vehicle', 'economy,', 'writes', 'thomas', 'marathon', 'promised', 'supposed', 'networks.', "thailand's", 'banned', 'targeting', 'limited', 'rapidly', 'way.', '16', 'instagram', 'twitter.', 'bear', "who's", 'facility', 'box', 'months.', 'outside', 'applications', 'issue', 'ten', '2016.', 'tourism', 'represent', 'group,', 'window', 'deliver', 'issued', 'camp', 'melania', 'revealed', 'communications', 'hired', '18', 'feeling', 'points', "tonight's", 'debate.', 'huge', 'margin', 'senator', 'cheaper', 'antigen', 'tests', 'losing', "haven't", 'taxes.', 'NBC', 'opponent', '911', 'york,', 'prosecutors', 'debates', 'empty', 'isolation', 'pain', 'racist', 'photo', 'driven', 'moon', 'maintain', 'trick-or-treating', 'accounts', "russia's", 'pose', 'remember', 'although', 'rejecting', 'placed', 'higher', 'design', 'orange', 'hospitalizations', 'outbreaks.', 'crowds', 'phones', 'faster', '5G', 'europe', 'package', 'chain', 'leader,', 'settlement', 'smaller', 'rival', 'seats', 'saudi', 'uses', 'attempting', 'create', 'investigate', 'issue.', 'declined', 'v.', 'properly', 'highly', 'rely', 'dogs', 'europe,', 'sick.', 'kidnap', 'detained', 'desperate', 'late-stage', 'paused', 'decades,', 'thanks', 'jean', 'remembered', 'turmoil', 'exposed', 'ohio', 'wanted', "i'm", 'revealing', 'china,', 'suburban', 'state.', 'grow', 'length,', 'mouth', 'consume', 'interview', 'london.', 'pleaded', 'guilty', 'ruth', 'bader', 'ginsburg', 'spots', 'defeating', 'robert', 'nobel', 'theory', 'era', 'connected', 'sharing', 'landmark', 'filled', 'completely', 'republicans', 'fired', 'city.', 'normally', 'thanksgiving', 'away,', 'poised', 'reports.', 'dreams', 'museum', 'alone', 'easy', 'returned', 'performances', 'damaging', 'health.', 
#  'insisted', 'rally.', 'come.', 'blocked', 'receiving', 'concluded', 'airborne', 'followed', 'happens', 'exchange', 'diplomats', 'on.', 'act,', 'city’s', 'tourist', 'add', 'acres', 'seeks', 'moral', 'removed', 'videos', 'green', 'says,', 'rich', 'upheld', 'driving', 'minister,', 'replaced', 'tools', 'east', 'Some', 'saturday', 'delta', 'unprecedented', 'prominent', "what's", 'stake', 'journalist', 'reduced', 'processing', 'built', 'eating', 'illness', 'death.', 'experimental', 'experiences', 'feel', 'visiting', 'rapper', 'los', 'angeles', 'deaths.', 'protecting', 'experienced', 'addressing', 'barriers', 'laws', 'daniel', 'details', 'brutal', 'modern', 'increasing', 'atmosphere.', 'gender', 
#  'QAnon', 'netflix', '$2', 'attempt', 'serve', 'reminder', 'I\'m', 'literature', 'individual', 'defended', 'harris', 'history', 'funds', 'goal', 'finish', 'urge', 'celebrate', 'soaring', 'concept', 'vice', 'farm', 'positions', 'guy', 'battling', 'lung', 'november', 'government.', 'facebook', 'weed', 'britain,', 'ET', 'billionaire', 'steve', 'generations', 'old', 'subject', 'cost', 'argue', 'undermine', 'dangerous', '2', 'developing', 'method', 'trust', 'weighing', 'decade', 'studies.', 'speakers', 'convention', 'tasked', 'throne', 'speech', 'becomes', '2021,', 'viral', 'sharply', 'chairman', 'joint', 'gen.', 'van', '"If', 'university.', 'australian', 'mainland', 'surging', 'smith', 'in.', 'fought', 'vaccine.', '2022.', 'needed.', 'newsletter:', 'side', 'tough', 'introduced', 'lab', 'Joe', 'present', 'too.', 'nominee', 'incident', 'organizations', 'jobs,', 'severity', 'protocols,', 'contracting', 'confident', 'seat', 'burned', 'press', 'misleading', 'wrong.', 'gun', 'die', 'either', 'birth', 'denied', 'him,', 'party.', '•', 'breaks', 'ground.', 'charles', 'shark', 'others.', 'society', 'develop', 'leadership', 'banks', 'feed', 'accepted', 'quite', 'say,', 'stronghold', 'raises', 'title', 'inner', 'circle', "who've", 'matter', 'fell', 'ill', 'possibly', 'win.', 
#  "they'll", 'proving', 'valley', 'bars', 'missed', 'work,', 'fall.', 'sense', 'leader.', 'minutes', 'ambulance', 'needed', 'E.', 'contest', 'complications', 'screening', 'increasingly', 'lady', 'navy', 'direction', 'conduct', 'yorkers', 'rarely', 'resume', 'midst', 'everything', 'extended', 'hitting', 'forest', 'respond', 'findings', 'CNN', 'struggle', 'word', 'contracted', 'quietly', 'experiencing', 'lisa', 'parole', 'unable', 'Here', 'drew', 'worldwide', 'afternoon,', 'attempted', 'proud', 'allowing', 'sit', 'maine', 'advantage', 'learning', 'employee', 'then,', 'deemed', 'serves', 'distribution', 'child', 'chrissy', 'husband,', 'loss', 'wrote', 'france,', 'greenhouse', 'fraud,', 'voting.', 'sick', 'brand', 'vatican', 'citing', 'strip', 'often', 'alexei', 'recovering', 'nerve', 'vladimir', '$10', 'infrastructure', 'initiatives', 'weigh', 'television', 'separate', 'italy', 'summer,', 'rocket', '545', 'doubled', 'tells', 'aside', 'parent', 'directly', 'survived', 'wednesday.', 'responding', 'analyzed', 'here’s', 'taste', 'certainly', 'am', 'woman,', 'For', 'challenger', 'accusations', 'britain’s', 'articles', 'universal', 'dismiss', 'talk', 'dream', 'band', 'blow', 'urgent', 'sale', 'weekly', 'belarusian', 'alexander', 'populous', 'taxes', 'first-ever', 'glitches', 'crisis.', 'roughly', 'update', 'lines', 'courts', '–', 'breonna', 'raid', 'wildfires', 'destroyed', 'complain', 'taught', 'let', 'fatally', 'televised', 'fine', 'matters', 'ago', 'immunity', 'italian', 'planet', 'participate', 'hall.', 'cancer,', 'happen', 'At', 'attacks.', 'marriage', 'fires', 'famed', 'wine', "california's", 'county,', 'forcing', 'goes', 'lake', 'facts', 'onto', 'affected', 'en', 'deadline', 'clearly', 'pass', 'case.', 'pregnancy', 'scale', 'doors', 'herself', 'ministry', 'analysis:', 'divisions', 'generally', 'impossible', 'simple', "you're", 'teacher', 'emmanuel', 'retailers', 'posts', 'pacific', 'baby', 'sources', 'surpassed', 'january.', 'pounds', 'government’s', 'pledge', 'demonstrators', 'three-quarters', 'meaning', 'muslim', 'hair', 'religious', 'complex', 'russia,', 'investigations', 'evangelical', 'GDP', 'shutdowns', 'intentionally', 'supporting', 'regulatory', 'reporters', '"He', 'insurance', 'normal.', 'activists', 'lukashenko', 'briefly', 'that.', 'kinds', 'korea', 'troops', 'age.', 'transfer', 're-election.', 'chaos', 'promises', 'thousand', 'They', 'de', 'vote,', 'presidents', 'limiting', 'tourists', 'forced', "britain's", 'form', 'credit', 'xi', 'burning', 'mounting', 'actions', 'minority', 'highlights', 'dutch', 'australia', 'died,', 'derek', 'kentucky', 'unite', 'roberts', 'words', 'describe', 'german', 'treating', 'nation.', 'command', 'plenty', 'rain', 'awaiting', 'called.', 'encounter', 'april', 'courts.', 'payette', 'checks', '6', "family's", 'coffins', 'colleges', 'commitment', 'headed', 'journey', 'entry', 'extraordinary', 'legacy', 'carry', 'thing', 'march', 'mitt', 'romney', 'slammed', 'offices', 'life,', 'factors', "australia's", 'campus', 'painting', 'square', 'delivering', 'together.', 'tapped', 'constitutional', 'addition', 'interfere', 'NFL', 'connection', 'britain', 'unless', 'jersey', 'anxious', 'lack', 'cloud', 'jonathan', 'comedy', 'completing', 'sweep', 'enforcement', 'addressed', 'airs', 'fundamental', 'cars', 'parking', 'crowd', 'afghan', 'hangs', 'for.', 'politician', 'immediately', 'edition', 'film', 'guidance', 'resource', 'outstanding', 'casting', 'july,', 'cyber', 'else', 'lots', 'disabilities', 'treasury', 'indian', 'Now,', 'yet,', 'prestigious', 'jim', 'worked', 'risk.', 'ambitious', 'unfounded', 'fraud.', 'wish', 'provided', '50,000', 'barr', 'bottle', 'blamed', 'politicians', 'One', 'help.', 'outlook', 'possibility', 'threat', 'tests,', 'positive.', 'trip', 'activist', 'agenda', 'anybody', 'count.', 'populations', 'ultimate', 'hunter', 'sometimes', "we've", 'china.', 'ran', 'man.', 'plane', 'max', 'knowing', 'gulf', 'panel', 'audio', 'flooding', 'city,', 'slowed', 'source', 'representative', 'championship', 'allegedly', 'smoke', 'covering', 'reforms', "taylor's", 'beirut', 'protects', 'diseases', 'types', 'out.', 'you.', 'commander', 'universities', 'twice', 'supported', 'operating', 'established', 'debt', 'sees', 'deeply', 'coach', 'viewed', 'found,', 'cemetery', '2020,', 'governors', 'week’s', 'hurricane,', 'center', 'atmosphere', 'deliberately', 'know,', 'analysts', "company's", 'wildfire', "Here's", 'stretch', 'boat', 'graham', 'bureau', 'discarded', 'safer', 'returning', 'weapons', 'resignation', 'conditions', 'deleted', 'exploring', 'efficient', '2019,', 'bin', 'al', 'disposable', 'boston', 'creative', '2019', 'fix', 'roles', 'sites', 'register', 'catching', 'marking', 'bold', 'similar', 'fate', 'dangers', 'husband', 'seems', 'aggressive', 'reverse', '75', 'december', 'march.', 'afghanistan', 'greece', 'founder', 'wipes', 'string', 'survivors', 'extreme', 'target', 'classified', 'india', 'sexually', 'her.', '1960s', 'proved', 'promoted', 'emerged', 'defend', 'fire,', 'california,', 'spooky', 'stars', 'successful', 'scientist', 'towns', 'tracking', 'improve', 'initially', 'reported,', 'finds', 'pace', 'refusal', 'throw', 'swept', 'ranks', 'gone', 'encourage', 'screen', 'shifted', 'trump’s', 'quick', 'cause', 'standing', 'understand', 'stocks', 'hood', 'online.', '24.', 'catholic', 'aboard', 'All', 'racial', 'longest', "court's", 'parliamentary', 'rape', 'art', 'charlie', 'institute', 'killed,', 'CDC', 'feature', 'emotionally', 'importance', 'football', 'began,', 'may.', 'didn’t', 'brief:', 'features', 'severely', 'draws', 'catch', 'communities', 'minds', 'themselves', 'hours.', 'ship', 'stark', 'blood', 'barely', 'planning', 'patience', 'thin', '45', 'weekend,', 'yes,', 'everyone', 'tonight.', 'live.', 'stickers', 'crew', 'signal', 'amazon,', 'quarter', 'february,', 'archaeologists', 'safely', 'familiar', 'code', 'lets', 'becoming', 'disaster', 'balance,', 'reform', '85-year-old', 'brooklyn', 'comments', 'restrained', 'heads', 'cruise', 'storage', 'stolen', 'jurors', 'content', 'hole', 'solar', 'cook', 'doubt', 'vaccine,', 'trials', 'metropolitan', 'know:', 'drive-thru', 'announced.', 'sizes', 'surrounded', 'jacob', 'volunteer', 'eric', 'instead.', 'house,', 'baron', 'degree', 'shrinking', 'statements', 'deal.', 'gains', 'camera', 'midwest', 'harry', 'eye', 'Read', 'more:', 'seemingly', 'temporary', 'page', 'struck', 'recession', 'italy,', 'common', 'happy', 'touched', 'far-right', 'piece', 'covid-19', 'blasio', "america's", '@CNNOpinion', 'co-founder', '#SilenceIsNotAnOption', 'podcast:', 'covid-19,', 'homeland', 'resign', 'banning', 'controlled', 'rep.', 'alaska', 'president-elect.', 'johns', 'hopkins', 'closely', 'texas,', 'grim', 'counties', 'arizona', 'project.', 'state’s', 'stood', 'US.', 'truth', '@donlemon', 'tweets', 'potentially', 'covid-19.', 'chicago', 'course', 'On', 'cuomo', 'thoughts', '🎧', 'listen:', 'recognize', 'GOP', 'pentagon', 'defense', 'inauguration', '--', 'today’s', '@DrSanjayGupta', 'McConnell', 'reject', 'montana', 'threatens', 'elderly', 'adults', 'census', 'adviser', 'campaign,', 'acknowledge', 'baseless', 'iPhone', 'mini', 'us,', 'ET/PT', 'passing', 'biden’s', 'decline', 'suggesting', 'divisive', 'fueled', 'americans,', 'refuse', 'victory,', 'guests', '📩', 'inbox', 'daily:', 'today:', 'UK', 'brexit', '@lukemcgee', 'win,', 'wild', 'talking', '@StCollinson', 'tennessee', 'sworn', 'primary', 'shanghai', 'entirety', 'relocated', 'dubbed', '"walking', 'machine."', 'virus,', 'protests,', '@jgriffiths', 'you,', 'W.', 'collapse', 'hurricanes', 'loved', 'ones.', 'board.', 'traveled', 'unlikely', 'CNN’s', 'labor', 'bernie', '“If', 'brad', 'discusses', '“We', 'statewide', 'confirm', 'Covid-19.', 'lost.', 'champion', '"President', 'placing', '@CillizzaCNN', 'el', 'paso', 'culture', 'results,', 
#  '20,', 'check:', 'baselessly', 'utah', 'india,', 'JUST', 'IN:', 'loeffler', 'perdue', 'traffic', 'navajo', 'all,', 'FDA', 'surrogate', 'McCain', 'cindy', 'regime', "CNN's", 'writes.', 'illinois,', 'academy', 'nationally', 'grown', '@ForecasterEnten', 'revenge', 'morning,', 'first:', 'warren', 'We', 'victory.', 'authorized', 'drugmaker', 'ceremony', 'event.', 'confederate', 'traveling', 'PROJECTION:', '#CNNElection', 'context', 'rampant', 'devastating', 'dominated', 'women,', 'mexico,', 'kong,', 'beijing', 'powers', 'mink', 'stacey', 'abrams', 'historically', 'graduates', 'producing', 'maryland', 'featuring', 'hall,', 'discover', 'check', 'whip', 'protections', 'correspondents', 'seriously', 'election:', 'la', 'adopted', 'fauci,', 'allergy', 'diseases,', 'pro', 'grounded', 'misinformation', 'cal', 'thom', 'dramatically', "democrats'", 'eventually', 'trump,"', 'surprise', 'globally', 'reacts', 'pair', 'recep', 'tayyip', 'erdogan', 'congratulated', 'updates:', 'posing', 'capture', 'runoff', 'performed', '"Jeopardy!"', 'way,', '“I', 'can’t', 'citizens', 'mitch', 'chamber', 'behavior', 'republican,', 'Biden-Harris', "team's", 'jr.', 'illinois', 'florida,', 'via', 'broadcast', 'invalidate', 'prepares', 'administration.', '"What', 'next,', 'dana', 'waves', 'II', '(and', 'iconic', 'photographs', 'prosecutor', 'far.', 'pressing', 'striking', 'gay', 'trusted', 'egypt', 'trebek,', '90%', 'effective,', 'polarized', '@CNNOpinion.', 'urban', 'carson', 'atlanta,', 'recreational', 'marijuana', 'disappearing', 'watched', 'no.', "harris'", '50%', 'confront', 'eli', 'firing', '@JohnKingCNN', 'podcast', 'fastest', 'jon', 'ossoff,', 'books', 'it,"', 'congress.', '“It', 'effectiveness', 'incumbent', 'sewage', 'covid', 'jared', 'k-pop', 'base', 'investigative', 'collins', 'latino', 'differences', 'latinos', 'life-threatening', 'defeat', 'heal', 'america?', 'devoted', '"You', 'color', 'democrat,', 'enten', 'landscape', '...', 'unity', 'kushner', 'biden:', 'favorite', 'fan', '"For', 'message.', 'office,', '"When', 'hometown', 'wilmington,', 'Watch', 'Follow', 'harris,', '“This', 'career.', 'tweeted', '“You', 'delaware,', '“It’s', 'nation’s', '46th', 'meadows,', 'helps', 'decide', 'meadows', 'know.', 'trajectory', 'suggests.', 'counted,', 'madison', 'records.', 'congressional', 'sarah', 'transgender', 'explains', 'freedom', 'deadlines', 'correct', 'ballot,', 'martha', 'votes,', 'christie', 'edge', 'stands', 'hasn’t', '▪︎', 'nevada', 'carolina', 'alabama', 'focusing', 'cori', 'bush,', 'here.', 'missouri', 'expectations', 'far,', 'breaking', '“The', 'USPS', 'color.', 'what’s', 'topic', 'shaping', 'unemployment', 'fort', 'department.', 'jump', 'YouTube', 'unpopular', 'oklahoma', 'warnings', 'reelection', 'election-related', 'podcast,', 'wall"', 'labeled', 'legalize', 'jersey,', 'susan', 'dakota', 'delaware', 'updates.', '#CNNelection', 'absentee', 'abortion', 'undecided', 'hawaii', 'nebraska,', 'alexandria', 'Ocasio-Cortez', 'iowa', 'flag', 'colorado', 'proposition', 'lindsey', 'jaime', 'hampshire', 'marjorie', 'LGBTQ', 'houston', 'wait.', 'performance', 'other,', 'here!', 'winning.', 'they’re', 'america,', 'deborah', 'alarming', 'disinformation?', 'lives,', "night's", 'chancellor', 'man,', 'interactive', '-', 'america’s', 'economy?', 'again,', 'ant', 'questioning', 'tuesday.', 'voting,', 'park,', 'approaches', 'materials', 'unrest', 'diverse', 'surprised', 'backlash', 'monarchy', '"But', 'civic', 'profound', 'african', 'chapter', 'electors', 'districts', 'favor', 'eta,', 'assailant', 'Day.', 'horrific', 'sentenced', 'questioned', 'players', 'candidates,', 'solo', 'carolina.', '@JohnAvlon', 'immigrant', 'consistently', 'rallies', 'feet', 'opinion', 'focused', 'forgot', 'surge,', 'catastrophic', 'celebrated', 'atlanta', 'citizenship', 'systems', 'wolf', "university's", 'pledging', "saturn's", 'molecule', 'measuring', 'sail', 'terrorist', 'lightning', 'sounds', 'bats', 'naturally', 'little-known', 'automaker', 'snatched', 'walter', 'wallace', 'pepper', 'spray', '@MaeveReston', '1918', 'worst,', 'flu.', 'scientific', 'tall', 'relationship,', 'alliance,', 'approaches,', 'seize', 'US-Mexico', 'wall,', 'digestive', 'feces.', 'Cities', 'hornets"', 'captured', 'kate', 'rubins', 'clinton', 'interview.', 'run,', 'ventilation', '99%', 'viruses,', 'harvard', 'accepting', 'travelers', 'gloves', 'connery', 'beverages', 'heart', 'origin', 'reportedly', 'administration,', 'memory', 'black,', 'nancy', 'sue', 'ohio,', 'Friday.', 'marijuana.', 'fracking', 'acquired', 'pete', 'buttigieg', 'prior', 'enter', '“He', 'shape', 'danger', 'photographer', 'jinping', 'jeffrey', 'hackers', 'scheme', 'percentage', 'competitive', 'racing', 'right,', 'pivotal', 'iowa,', 'fatal', 'place,', 'brett', 'hoped', 'pre-election', 'struggles', 'handing', 'milky', 'facebook.', 'czech', 'unexpected', 'surpassing', 'ballot.', 'xinjiang,', 'In', 'lasting', 'transmit', 'answer', 'speaker', 'pelosi', 'consequential', 'If', 'adds.', 'pres.', 'battlegrounds', 'near-Earth', 'fit', 'speaks', 'profit', '14%', 'jr.,', 'scandal', 'contentious', 'facebook,', 'perhaps', '“not', 'aren’t', 'hispanic', 'Here’s', 'theater', 'first-time', 'google,', 'decapitated', 'he’s', 'we’re', 'conclude', 'dodgers', 'kavanaugh', 'jet', 'substantial', 'partial', 'CIA', 'disputes', 'tomorrow', 'upper', 'canceled', 
#  'cultural', 'war,', 'all-new', '#FirstLadies,', 'soccer', 'turnout.', 'sacha', 'tampa', 'rays', 'firefighters', 'younger', 'point,', 'i’m', 'folks', 'reaction', 'classic', 'likes', 'apologized', 'don’t', 'pandemic:', 'audience', 'Biden.', 'latin', 'he’ll', 'solid', 'roosevelt', 'comply', 'emily', 'chronic', '@ddale8', 'magazine', 'columnist', 'firmly', 'uyghurs', 'motivation', 'justify', 'shield', 'republicans.', 'vietnam', 'stance', 'article', 'we’ve', 'peak', 'paper', 'republicans,', 'boycott', '14-year-old', '$25,000', 'Anika', "chebrolu's", 'invention', 'in-silico', 'methodology', 'selectively', 'bind', 'protein', 'SARS-CoV-2', 'pence’s', 'challenger,', 'last-ditch', 'policy.', 'located', 'insufficient', 'render', 'ineffective,', 'specialist', 'peru,', 'phrase', 'lesley', 'eleanor', 'elevate', 'instead,', 'African-American', 'diet', 'tea', 'pressure,', 'bigger', 'trove', 'ancient', 'women.', '4.', 'randi', 'kaye', 'founded', 'disasters', 'anger', 'luck', 'year’s', 'racially', 'francisco', 'stadium', 'it,”', '“They', 'rule', '“A', 'McConnell,', 'editors', 'enters', 'With', 'plotting', 'predominantly', '60%', 'pushed', 'writer', '#Debates2020', 'topics', 'times,', "thursday's", 'colorado,', 'typical', 'customs', 'undocumented', 'lowest', 'won’t', 'lagos', 'official,', 'worrying', 'transformed', 'scattershot', 'insult', 'products.', 'abruptly', 'zion', 'suburbs', 'isn’t', 'exploded', 'epsilon', 'referred', '6%', 'paint', 'pursue', 'worsened', 'openly', 'boxes', 'harnessing', 'agents', 'kansas', 'lay', 'person?', 'kim', 'trumpism', 'suburbs.', 'periods', 'Thursday.', 'tries', 'repeated', 'vowed', 'reagan', 'See', 'guide', 'all.', 'averaging', 'remarkable', 'interviews', 'QAnon,', 'likelihood', 'songs', 'electricity', 'tallies', 'live,', 'europe.', 'nick', 'saban', 'she’s', 'heated', 'president’s', 'giuliani,', 'films', 'dueling', 'ABC', 'complicated', '“a', 'you’re', 'editor,', 'economist', '“I’m', 'gaming', 'them.”', 'barrett’s', 'there’s', 'news:', 'flipped', 'county:', 'que', 'un', 'del', 'presidente', 'talked', 'percent', 'daily.', 'china’s', 'hundred', 'prompting', 'majority.', 'FTI', 'scenes', 'cracking', 'world’s', 'Opinion', 'President', 'threw', 'georgia’s', 'armistice', 'aung', 'suu', 'harris’s', 'paths', 'lael', 'party’s', 'defeat,', 'como', 'una', 'y', 'para', 'errors', 'maricopa', 'latest:', 'swung', 'unsubstantiated', 'why?', 'europe’s', 'defeated', 'weakness', 'democrats’', 'france’s', 'democrats.', 'habitable', 'whatever', 'asks', 'live:', 'wasn’t', 'early,', 'you’d', 'decisive', 'falsehoods', '#Election2020', 'plumbing', 'partisan', 'midwestern', 'Miami-Dade', 'people’s', 'Eastern', 'demographic', 'india’s', '#ElectionDay', 'newsletter', '#election2020', 'editor', 'administration’s', 'africa', 'implementing', 'writes,', 'Times/Siena', '“long', 'working-class', 'forecasting', '🗳', 'thursday’s', 'fridge', 'college-educated', 'dollar', 'Find', '📌', 'africa’s', 'monopoly', 'peru', 'value', 'bound', 'inflame', '@1843mag', 'assesses', '“Money', 'talks”', '@AnneMcElvoy', 'asks”', 'roll-out', '@tomstandage', 'ahead”', '“Checks', 'balance”', '-@jonfasman', '-@gelliottmorris', '-@DSORennie', 'intelligence”', 'booze', '“Babbage”', 'rigorous', 'fair-minded', 'intelligence”:', "economist's", 'circulation"', 'involve?', 'OpenRAN', "kelly's", 'all-Democratic', 'trot', 'front-runners', 'sistine', '@projectlincoln', '@NHJennifer', '@KnCukier', '@T_Wainwright', 'galicia', 'nigel', 'mr', 'intelligence”,', 'editor-in-chief', 'hub', '@jonfasman', 'tigray', 'economist’s', 'ahmed', 'treasuries', 'mapped', '“Democracies', 'partisan.', 'unique...But', 'tribalised.”', 'partisanship', '@BoFrankln', 'voted?', 'ethiopia’s', 'fake.', '"Babbage"', 'wandered', "philadelphia's", '“working', 'hotel”']

# Trump words list
# words_list = ['be', 'sure', 'to', 'tune', 'in', 'and', 'watch', 'donald', 'trump', 'on', 'late', 'night', 'with', 'david', 'letterman', 'as', 'he', 'presents', 'the', 'top', 'ten', 'list', 'tonight!', 'will', 'appearing', 'view', 'tomorrow', 'morning', 'discuss', 'celebrity', 'apprentice', 'his', 'new', 'book', 'think', 'like', 'A', 'champion!', 'financial', 'show', '-', 'very', 'finale', 'lessons', 'learned', 'along', 'never', 'that', 'of', 'a', 'rather', 'build', 'than', '--Donald', 'J.', 'miss', 'USA', 'not', 'fired', '"I\'ve', 'always', 'been', 'second', 'says', 'listen', 'an', 'interview', 'discussing', 'book,', 'for', 'keep', 'your', 'sense', 'wonder', 'enter', '"Think', 'signed', '"When', 'it\'s', 'plateau,', 'it’s', 'afraid', 'being', 'unique', 'best', '"We', 'win', 'our', 'lives', 'by', 'having', 'each', 'could', 'all', 'use', 'little', 'power', '"Always', 'know', 'you', 'something', 'read', 'great', 'york', 'times', 'it', 'fast,', 'short', 'direct', 'whatever', 'don\'t', 'forget', 'did', 'is', 'become', 'fan', 'today!', 'higher', 'self', 'opposition', 'comfort', 'from', 'champion', 'trump:', '"You', 'have', 'when', 'call', 'moving', 'check', 'out', 'trump\'s', '“If', 'problems,', 'you\'re', 'pretending', 'or', 'run', 'own', 'business.”', 'last', 'week', 'today', 'send', 'him', 'wishes', 'here:', 'thanks', 'birthday', '–', 'vanity', 'was', 'one', '&', 'trump."', 'wishing', 'happy', 'father\'s', 'day', 'there', 'YOU', 'are', 'life', 'every', 'trump’s', 'WWE', 'does', 'big', 'michael', '"He', 'friend', 'spectacular', 'devastating', 'powerful', 'frank', 'about', 'economy', 'greta', 'van', 'here\'s', 'safe', 'enjoy', 'it!', '"Be', 'aware', 'things', 'seem', 'because', 'they', 'can', 'step', 'towards', 'congrats', 'winners', 'around', 'world', 'who', 'entered', 'books', 'summer', 'reading', 'at', 'university', 'business', 'success', 'that’s', 'first', 'develop', 'ivanka', 'now', 'twitter', 'follow', 'her', '@IvankaTrump', 'terrific', '--', 'recent', 'appearance', 
#               'universe', 'competition', 'LIVE', 'sunday,', '@', '9pm', 'tonight', 'buy', 'double', 'out:', 'more', 'featuring', 'trump"', 'plus', 'golden', 'race', '"A', 'lot', 'people', 'but', 'can\'t', 'what', 'has', 'say', 'daughter', 'upcoming', 'video', 'sharing', 'advice', '4', 'women', 'hear', 'gov', 'spending,', 'taxes', 'wedding', 'make', 'beautiful', 'i’m', 'proud', 'ahead', 'greatest', 'golf', 'course', 'international', 'scotland.', 'reality', 'series', 'ultimate', 'yours', 'tower', 'chicago', 'ranked', 'tallest', 'building', 'council', 'buildings', 'everyone', 'wonderful', 'holiday', 'prosperous', 'year.', 'let’s', 'champions', 'returns', 'NBC,', 'outstanding', 'celebrities', 'season', 'should', 'yet!', 'app', 'iPhone', 'touch', '"I', 'saw', 'lady', 'she', 'channel', 'host', 'match', 'CA', 'mark', 'vs.', 'kevin', 'sunday', 'american', 'already', 'may', 'team', 'win!', 'CNN', 'submit', 'final', 'episode', 'jay', 'deliver', 'special', 'so', 'my', 'wife', 'melania', 'launch', 'debut', 'QVC', 'april', '9', 'Mar-a-Lago', 'many', 'entertainment', 'long', 'history', 'palm', 'beach', 'premiere', 'this', 'march', 'p.m.', 'NBC.', 'see', 'off', '10', 'hour', 'key', 'olympic', 'gold', 'just', 'left', 'office.', 'town', 'wanted', 'meet', 'guy', 'true', 'champion.', 'I', 'am', 'announce', 'offer', 'job', 'opportunities', 'those', 'coming', 'back.', 'week\'s', 'set', 'stage', 'season.', 'NBC', 'even', 'hotel', 'collection', 'currently', 'nominated', 'conde', 'nast', 'choice', 'travel', 'original', 'takes', 'next', 'NYC', '1', 'tower.', 'information', 'go', 'open', 'apprentice.', 'look', 'forward', 'seeing', 'there.', 'today\'s', 'drew', 'thousands', 'impressive', 'enjoyed', 'meeting', 'them.', 'we\'ve', 'got', 'some', 'attended', 'skating', 'stars', 'gala', 'rink', 'central', 'fantastic.', 'stay', 'part', 'then', 'we', 'scottish', 'fashion', 'benefits', 'which', 'sean', 'available', 'read.', 'opens', 'friday', 'fantastic!', 'downtown', 'york.', 'unlike', 'anything', 'else.', 'visit', 'fantastic', 'soon!', '"Donald', 'fabulous', 'ET', 'looking', 'yankees', 'opening', 'day!', 'kids', 'need', 'place', 'police', 'organization', 'i\'m', 'support', 'play', 'int\'l', 'against', 'panel', 'experts', 'adam', 'great.', 'amazing', '11', 'a.m.', 'CBS.', 'fun!', 'put', 'live', 'las', 'vegas', '7', 'i\'ll', 'show!', 'thoughts', 'prayers', 'remain', 'bret', 'michaels', 'family', 'recovery.', 'larry', 'king', 'tonight,', 'CNN.', 'pm', 'where', '2010', '16', 'hosted', 'stone', 'also', 'pageant', 'country', 'rock', 'commentary', 
#               'joan', 'we\'re', 'progress', 'full', 'vegas.', 'congratulations', 'represent', 'us', 'well', '.', 'page', 'up', 'what\'s', 'apprentice,', 'free', 'tickets', 'latest', 'he\'s', 'making', 'effort', 'attend', 'in!', 'who\'s', 'vote', 'facebook', 'were', 'caught', 'both', 'way.', 'ratings', 'had', 'time.', 'everyone!', 'scotland', 'beautiful.', 'spent', 'years', 'right', 'place,', 'visiting', 'over', '200', 'absolutely', 'place!', 'north', 'coast', 'rolling', 'dunes', 'here', 'they\'re', 'how', 'study', 'movement', 'trip', 'memorial', 'thought', 'veterans', 'done', 'much', 'turning', 'saturday', 'night.', 'honor', 'anniversary', 'show.', 'checking', 'CNN,', 'trade', 'places', '9PM', 'honored', 'chosen', 'line', 'their', 'NY', 'ride', 'fame', 'campaign.', 'ribbon', 'cutting', 'ceremony', 'front', 'site', 'me', 'covers', 'dinner', 'hotel.', 'hottest', 'restaurants', 'city.', 'hotels', 'TV', '"The', 'only', 'omarosa', 'brave', '...', 'relief', 'tonight\'s', 'two', '8', 'million', 'raised', '2', 'hours.', 'eric', 'foundation', '$1,000,000', 'st.', 'children\'s', 'research', 'shots', 'article', 'wall', 'street', 'work', 'expected', 'begin', 'weekend.', 'for.', 'due', 'popular', 'demand,', 'monday', 'july', 'fall', 'putting', 'back', 'work!', 'george', 'steinbrenner', 'anyone', 'lost', 'truly', 'man.', 'visited', 'courses', 'yet', 'another', 'them', 'accept', 'humanitarian', 'award', 'helping', 'counting', 'get', 'work.', 'august', 'performing', 'bay', 'resort', 'john', 'september', '14', '14th', 'event', 'held', 'national', 'club', '3', 'hire', 'would', 'thursday', 'season!', 'four', 'days', 'until', 'pageant.', 'going', 'there!', 'mexico,', 'ratings!', 'everyone.', 'weekend', 'went', 'game', 'bill', 'time', 'watching', '11:30', '(and', 'invited', 'sit', 'box', 'during', 'U.S.', 'yesterday.', 'she\'s', 'there\'s', 'thursday,', 'air', 'fall.', 'excitement', 'opportunity.', 'annual', 'charity', 'him.', 'EST', 'action', 'packed', 'exciting', 'dog', 'night\'s', 'weekly', 'doctor', 'administration', 'robert', 'aberdeen,', 'well.', 'won\'t', 'forget!', 'solutions', 'company', '1,000', 'find', 'ideal', 'prepared', 'night,', 'morning.', 'talked', 'fox', 'news.', 'series.', 'want', 'surprise', 'susteren', 'good', 'info', 'about.', 'getting', 'down', 'action!', '"', 'shown', 'staff', 'sgt.', 'A.', 'received', 'medal', 'pres.', 'obama', 'month.', 'today.', '5', 'thanksgiving', 'featured', 'delivers', 'guy,', 'vote!', 'high', 'way,', 'apprentice!', 'clint', 'player.', 'awards', '.....', 'five', 'york,', 'vegas,', 'show,', '25', 'pm.', 'television', 'friend.', 'healthy', 'year!', 'jimmy', 'fallon',
#               'afternoon', 'speaking', 'neil', 'FOX', 'busy', 'doing', 'friends,', 'its', 'beginning', 'january', 'ET.', 'THe', 'right!', 'supposed', 'compete', 'china', 'other', 'no', 'environmental', 'workers', 'third', 'entire', 'piers', 'morgan', 'won', 'speech', 'CPAC', 'washington', 'DC', 'yesterday', 'comments', 'washington,', 'desk', 'asked', 'if', 'i\'d', 'interested', 'came', 'don', 'willing', 'answer', 'tweet', 'answered', 'questions', '6', 'TODAY', 'starts', 'you\'ve', '15th', '10:30', 'made', 'recently', 'developing', 'towers', 'republic', 'georgia.', 'fireworks', 'hey', 'real', 'fazio', 'talking', 'most', 'catch', 'news', 'now,', '@GMA', 'report', 'CNBC', 'we\'ll', 'current', 'politics.', 'in.', '#1', 'highest', 'rated', 'brought', 'viewers', 'spoke', 'friends', 'lots', 'better', 'lives.', 'really', 'do', 'through', 'ad', 'money', 'gave', 'response', 'baseball', 'O\'Reilly', 'factor', '8pm', 'you\'ll', 'press', 'conference', 'car', '@todayshow', 'interviewed', 'case', 'missed', 'answering', 'tweets', 'please', '#trumpvlog', 'republicans', 'out,', 'democrats', '@hannityshow', 'tea', 'party', 'past', '#trumpvlog...', 'working', 'yesterday\'s', 'events', 'sunday\'s', 'congratulate', 'president', 'men', 'armed', 'forces', '(cont)', 'hampshire', 'sneak', 'exclusive', 'we’ve', 'center,', 'trump.', 'difficult', 'decision', 'regarding', 'presidential', 'thank', 'fans', 'support.', '#MissUSA', 'ready', '@foxandfriends', 'rich', 'throughout', 'great!',
#               'tremendous', 'few', 'months,', 'tuned!', 'day,', 'soldiers', 'protected', 'country.', '@SarahPalinUSA', 'anthony', 'weiner', 'why', 'holding', 'favorite', 'USA!', 'telecast', 'among', 'ABC,', 'huge', 'success.', 'i\'ve', 'links', 'anywhere', 'world.', 'office', 'san', '20,000', 'people.', 'issues', 'estates', '@EricTrump', 'celebration', 'panama', 'perhaps', 'kind', 'thinking', 'congress', 'TO', 'AND', 'debt', 'limit', 'cannot', 'spending', 'negotiate', 'themselves', 'again', '@BarackObama', 'art', 'thing', 'deal', 'strength,', 'leverage', 'biggest', 'strength', 'focused', 'budget', 'borders', 'america', 'path', 'ObamaCare', 'tragedy', 'further', 'deficit', 'billion', 'medicare', 'fund', 'house', 'GOP', 'wants', 'cut', 'took', 'wrong!', 'derek', 'incredible', 'ocean', 'ceiling', 'el', 'while', 'tries', 'push', 'gun', 'control', 'still', 'project', 'US', 'named', 'take', 'these', 'deal"', '8PM', 'husband', 'birth', 'rose', 'must', 'act', 'let', 'dollar', 'spends', 'costs', 'interest', 'children', 'doesn\'t', 'waste', 'opportunity', 'obamacare', 'any', 'deal.', 'raise', 'low', 'nation', 'destroy', 'continues', 'jobs.', 'such', 'total', 'president.', 'tax', 'increase', 'economic', 'growth', 'cuts', 'into', 'record', 'proves', 'it.', 'balanced', 'spending.', 'gang', 'disaster.', 'REPEAL', '@gretawire', '"Our', 'leaders', 'leading', 'america’s', 'senator', '@BarackObama,', 'ronald', 'continued', 'attack', '$', 'lead', 'ever', 'rising', 'gas', 'prices', 'bad', 'amendment', 'fiscal', 
#               'social', 'security', 'growing', 'pressure', 'obama,', 'sadly,', 'private', 'sector', 'cash', 'costing', 'taxpayer', 'fraud', 'before', 'funding', 'situation', 'crazy', 'everything', 'america\'s', 'greater', 'killing', 'needs', 'toughest', 'fast!', 'kill', 'wild', 'government', 'waste.', '@QVC', 'her!', '"Know', 'walk', 'away', 'ask', 'pictures', 'works', 'maybe', 'newsmax', 'totally', 'her,', 'appreciate', 'words', 'mike', 'fine', 'swing', 'written', 'book.', 'definitely', 'worth', 'quit', 'hit', 'comeback', 'blue', 'commission', 'agree', 'future', 'fight', 'year', 'loses', 'details', 'watched', 'guy.', 'victory', 'america.', 'jobs', 'panels', 'problem.', 'pollster', 'pat', 'voters', 'sick', 'failure.', 'disappointed', 'mistake.', 'defense', 'strong.', 'help', 'strengthen', 'created', 'problems.', 'learn', 'deal,', 'worse', 'approved', 'politicians', 'wake', 'believe', 'proper', 'leadership.', 'pass', 'white', 'joe', 'stealing', 'statements', 'ideas', 'without', 'giving', 'credit', 'story', 'life!', 'answers', 'republican', 'record,', 'coal', 'fire', 'pizza', 'florida', 'enough', 'league', 'friday.', 'officially', 'became', '100%', 'GDP', '@BarackObama\'s', 'care', 'impact', 'economy.', 'result', 'increased', 'reckless', 'care.', 'this.', 'andy', 'radio', 'talk,', 'former', 'NOW', 'tell', 'vacation', 'oil', 'starting', 'rise', 'despite', 'horrible', 'times.', 'OPEC', 'rip', 'off.', 'leadership', 'fed', 'market', 'dollars.', 'wrong', 'debate', 'straw', 'poll.', 'leader.', 'mitt', 'romney', 'defend', 'debate?', 'easy', 'americans', 'career', 'create', 'believes', 'government.', 'ron', 'paul', 'wasting', 'trillions', 'dollars', 'iraq', 'afghanistan.', 'said', 'treasury', 'happy.', 'save', 'security,', 'reporters', 'buying', 'goes', 'up.', 'unemployment', 'black', 'hispanic', 'threat', 'example', 'wastes', 'owes', 'tells', '---', 'way!', 'personal', 'constantly', 'karl', 'rove', 'obama.', 'asking', 'give', 'EST.', 'israel', 'respond', 'told', 'so-called', 'rebels', '“We', '50%', 'complete', 'same', 'libya', 'europe', 'let\'s', 'start', 'celebrating', 'over.', 'comes', 'watch.', '$1', 'back?', 'solution', 'country\'s', 'problems', 'year\'s', 'election.', 'china\'s', 'corporate', 'leadership,', 'stopped.', 'deals', 'son', 'love', 'spend', 'agree.', 'hurricane', 'libyan', 'given', 'return', 'military', 'iran', 'admits', 
#               'letter', 'learn?', 'crude', 'isn’t', 'half', 'that.', 'ripping', 'terrorist', 'group', 'UN', 'unilaterally', 'actions', 'fully', 'small', 'turn', 'interests', 'reach', 'far', 'within', 'borders.', 'taken', 'islamic', 'EPA', 'energy', 'PM', 'stand', 'firm', 'immediately', 'defund', 'furious', 'ALL', 'way', 'house.', '2012', 'effect', '2014', 'expect', 'fast', 'hands', 'mexican', 'drug', 'killed', 'price', 'compared', 'laughing', '@BarackObama.', 'running', 'now!', 'fix', 'selling', 'guns', 'tough', 'community', '@IngrahamAngle', 'speeches', 'wouldn\'t', 'record.', 'plans', 'borrow', 'billion.', 'planning', 'ignore', 'iran\'s', 'nuclear', 'POTUS', 'old', 'jobs?', 'companies', 'US.', 'macy’s', 'herald', 'square', 'wednesday', 'celebrate', 'home', 'program', 'stopped', 'means', 'necessary.', 'forum', 'fifth', 'competitive', 'fallen', 'under', '2011', '#MissUniverse', 'grow', 'didn’t', 'threatening', 'regulations.', 'approve', 'tomorrow!', 'bob', 'message.', 'keeps', 'taxes.', 'growth.', 'continuing', 'openly', 'lost.', 'macy\'s', 'department', 'win.', '@TeamCavuto', 'border', 'murder', 'come', 'straight', '@FoxNews', 'wait', 'solyndra', 'photo', 'loves', 'money.', 'imagine', 'passed', 'protecting', 'manufacturing', '@netanyahu', 'announcing', '"It', 'cost', 'emmys', 'natural', 'resources', 'independent.', 'instead', 'continue', '....', 'concerned', '@Israel', 'closely', 'raising', 'seems', 'problem', 'reason', 'hard', 'speeches.', 'clue.', 'sorry', 'go.', '"If', 'rick', 'perry', 'stop', 'neither', 'ally', 'beat', 'abuse', 'solve', 'crisis', 'term.', 'couldn\'t', 'faith', 'freedom', 'orlando', 'sent', 'china’s', 'oil.', 'green', 'energy.', 'after', 'increases.', '@MittRomney', '@CNN', '@DonaldJTrumpJr', 'toronto', 'used', 'americans.', 'now.', 'jewish', 'friends.', '@seanhannity', 'completely', 'repealed.', 'shows', 'number', 'makes', 'proud.', 'demand', 'currency', 'end', 'rampant', 'chuck', 'tim', 'present', 'actual', 'games.', 'performance', 'event.', 'O\'Donnell', 'him!', 'meetings', '@GovChristie', 'position', 'simply', 'policies', 'face', 'iranians', 'threatened', 'laugh', 'us.', 'allow', 'code', 'incapable', 'anything.', 'boycott', 'billions', 'solar', 'numbers', 'letting', 'fair', 'rapidly', 'dying', 'magazine.', 'hopefully', 'atlantic', 'ever.', 'news!', '@Gretawire', 'charlottesville,', 'grand', 'guy!', 'super', 'committee', 'wasteful', 'first.', 'gallon', 'doubled', 'bill.', 'immediately.', '$2', 'order', 'premium', 'repeal', 'ObamaCare.', 'nothing', 'disgrace.', 'primary', 'schedule', 'rebuilding', 'rebuild', 'cares', 'born', 'policy', 'spread', 'lower', 'talk', 'liberal', 'billionaire', 'release', 'interesting', 'numerous', 'governor', 'chinese', 'bill!', 'protesters', 'upset', 'major', 'midas', 'campaigning', 'followers.', 'passing', 'sooner', 'later.', 'announced', 'trillion', 'largest', 'history.', 'serious', 'security.', 'israeli', 'saudi', 'diplomatic', 'publicly', 'supporting', 'OWS', 'protests', 'home.', 'growth,', 'term', 'treating', 'mistakes,', 'blowing', 'incompetent', 'evening', 'people\'s', 'finding', 'close', 'advance', 'home!', 'leaders.', 'this?', 'promote', 'iranian', 'index', '28', 'high.', 'all-time', 'WWII', 'leaving', 'amount', 'oil!', 'iran.', 'plan', 'failed', 'one.', 'creating', 'radical', 'form', 'law', 'fair!', 'else,', 'sad', 'followers', 'calling', 'important', 'you!', 'election', 'nominate', 'candidate', '@Lawrence', 'false', 'me.', 'sue', 'him?', 'majority', 'candidates', 'rates', 'china.', 'move', 'aren\'t', 'wide', '@CNN,', '@ApprenticeNBC', 'taking', 'interesting.', 'nation\'s', 'worst', 'force', 'war', 'clear', '@piersmorgan', 'penalty', 'bernie', 'tony', 'la', 'jon', 'stewart', 'al', 'qaeda', 'flag', 'flying', 'benghazi.', 'global', 'warming.', 'hacking', 'ally.', 'agreed', 'includes', 'filed', '40%', 'problem,', 'taliban', 'troops', 'dangerous', 'muslim', 'brotherhood', 'debt.', 'cuts.', 'mac', 'helped', 'cause', 'weapons', 'interests.', 'occupy', 'cyber', 'respect', 'feels', 'reporter', 'boring', 'vision', 'living', 'extreme', 'better.', '"Time', 'federal', 'dream', 'deficits', 'caused', 'party.', 'defending', 'fail.', 'evening.', 'handled', 'bail', 'mess', 'problem!', 'loan', 'bankruptcy', 'prove', 'corrupt', 'debate,', 'endorsement', 'death', 'political', '2012.', 'arab', 'spring', 'turned', 'behind', 'day.', 'october', 'me,', 'afford', 'food', 'low.', 'fewer', 'failure', 'intelligence', 'bin', '“When', 'mistakes', 'be,', 'tough,', 'december', '@CNBC', '@SquawkCNBC', 'army', 'chief', 'seeking', 'ties', 'once', 'iraq.', 'months', 'called', 'employment', 'since', 'unless', 'delayed', 'keystone', 'pipeline.', 'gone,', '"is', 'net', 'negative', 'admitted', 'CBO', 'complaining', 'marines', 'placed', 'across', 'latin', '#TimeToGetTough', 'defeated', 'bankrupt.', 'hawaii', 'violence', 'trouble.', 'delaying', 'sale', 'aircraft', 'message', 'threats', 'ways', 'admitting', 'mistake', 'agreeing', 'projected', 'world\'s', 'succeed,', 'lead.', 'exactly', 'egypt', 'hot', 'protest', 'abandoned', 'dec', '5th', 'again.', 'amazon', 'actually', 'washington.', 'didn\'t', 'campaign,', 'golf.', 'critical', 'deadline', 'foreign', 'least', 'executive', '@TrumpSoHo', 'disclosure', 'worth.', 'sanctions', 'alone', 'russia', 'soon', 'borrowing', 'rate', 'islamists', '2%', 'recovery', 'businesses', 'confirmed', 'investment', 'public', 'drive', 'banks', 'slowing', 'trade.', 'produce', 'newly', 'released', 'emails', 'data', 'later', 'products', '#TimeToGetTough.', 'downgraded', 'outlook', 'why?', '@TrumpWaikiki', '2nd', '@FoxNews.', 'states.', 'sad!', 'stop.', 'speaker', 'week.', 'include', 'bold', 'plans.', 'wealth', 'though', 'NOT', 'met', '@newtgingrich', 'might', 'newsmax-@iontv', 'media', 'production', 'RT', '@Newsmax_Media:', 'heads', 'poverty', 'reagan', 'purpose', 'possible,', 'establishment', '@KarlRove', 'directly', 'responsible', 'losing', 'becoming', 'debate.', 'presidency', '@JoeNBC', '@morningmika', 'excited', 'sarah', 'terrific.', '#TimeToGetTough,', 'clip', 'organization.', 'jeffrey', 'endorsed', 'tuesday', 'playing', 'class', 'paid', 'ft.', 'pres', 'IA', '27', ',', 'signing', 'copies', 'hope', 'located', 'avenue,', 'between', 'looks', 'join', '11am', 'taxes,', 'debt,', 'dealing', 'defeat', 'explained', 'deserve', 'laden', 'crowd', 'express', 'iowa', 'congressman', '@SteveKingIA', 'strong', 'point', 'calls', '@RickSantorum', 'weeks', 'sad.', 'amy', '@megynkelly', 'clueless', '@oreillyfactor', '@jayleno', 'guys', 'network', 'negative.', 're', '15', '12', 'noon', 'mentioned', 'single', 'question', '@ABC', 'saturday.', 'debate!', 'trump,', 'folks', 'debates', 'out.', 'polls', '@theviewtv', 'dealmaker', 'house,', 'knows', 'smart', 'deals.', 'deal:', 'ground', 'destroyed', 'stupid', 'road', 'NY,', 'cancelled', 'gotten', 'pay', 'bring', 'avenue', 'B', 'e-mails', 'seek', 'elect', 'feel', 'rosie', 'whose', 'parents', 'loser.', '@Rosie', 'funny', 'focus', 'ratings,', 'mention', 'name', 'anymore', 'same.', 'explaining', 'keeping', 'options', '@RedState', 'oil,', 'read!', 'welfare', 'individuals', 'entitled', 'closing', 'much.', '17', 'couple', 'fellow', 'patriots', 'done,', '@JonHuntsman', 'rally', 'she’s', 'state.', 'acted', 'european', '@TIME', '\'The', 'person', 'lines', 'share', 'civil', 'mean', 'dead.', 'that\'s', 'exceptional', '@BarbaraJWalters', 'fascinating', 'piece', 'cars', 'china,', 'longer', 'weakness', 'setting', 'uses', 'excuse', 'christmas', 'less', 'away,', 'gift', 'loved', 'hall', '@ralphreed\'s', 'time!', '$4', 'plain', 'wrong.', 'healthcare', 'forbes', '30', 'job!', 'advisor', 'jim', 'jones', 'pipeline', 'so.', 'system', '"It\'s', 'requires', 'rules', 'apply', '@Forbes', 'idea', 'realize', '"People', 'ability', 'too', 'thinks', 'fact', 'pays', 'biden', 'statement.', 'tom', 'average', 'truth', 'bigger', 'WH', 'defended', 'enemy.', 'decide', '4th', 'winning', 'spot', 'california', 'looked', 'congress.', '3rd', 'leave', 'iraq,', 'he’s', '“The', 'that,', 'end,', 'aren’t', 'ones', 'foot', 'bill,', 'cents', 'state,', 'taxpayers', 'millions', 'struggling', 'capital', 'gain', 'dependent', 'middle', 'east.', 'TRUMP', '+', 'world’s', 'changed', 'voter', 'month', '20%', 'unemployment.', 'remember,', 'wasted', 'vote.', 'wins', 'credibility.', 'no.', 'hacked', 'expanding', 'expand', 'issued', 'statement', 'issue', 'projects', 'except', 'poor', 'work."', 'destroying', 'track', 'ballot', '50', 'predicted', '9/11', 'collapse', 'right,', 'russell', 'not.', 'sending', 'copy', 'hold', 'parade', 'returning', 'afghanistan', 'veterans.', 'plays', 'finish', 'dead', 'campaign', 'chairman', 'her.', '9/11.', 'finally', 'robbing', 'blind', 'defends', 'texas', 'caucus', 'primary,', 'lose', 'announcement', 'journal', 'reported', 'obama\'s', 'stamp', 'massive', 'crime', '"To', 'disloyal', 'terrible', 'offered', 'donation', 'supporters', 'papers', 'funded', 'winner!', 'votes', 'nice', 'zero', 'chance', 'gains', 'anything,', 'nations', 'require', 'clearly', 'quickly', 'falling', 'dropped', 'showed', 'poll', 'nobody', '(or', 'michelle', 'likes', 'excellent', 'warned', 'carrier', 'three', 'cast', 'field', 'simple', 
#               'beyond', 'understood', 'begins', 'driving', 'brady', 'covered', 'proven', 'herself', 'writer', 'iran,', 'countries', 'benefit', 'them,', 'whenever', '@marcorubio', 'found', 'jobs,', 'using', 'industry', 'donors', 'lunch', 'drain', 'treasure', 'potential', 'NH', 'tough.', 'understand', 'strong,', 'damage', 'democrats,', 'weak', 'disaster', 'winner.', 'field.', 'gives', 'spoken', 'manage', 'agenda.', 'embassy', 'kicked', 'right:', 'root', 'pride', 'saving', 'deserves', 'commander', 'respects', 'challenges', '10%', 'man', 'people!', 'ripped', 'gets', 'attacks', 'bain', 'mistake!', 'USMC', 'praised', 'protect', 'led', '"For', 'pushed', 'military,', 'negotiating', 'enemy', 'authority', 'well!', 'ratings.', 'cold', 'state', 'SC', 'lousy', '“Get', 'dollars,', 'bought', 'fighting', 'cancer', 'receiving', 'achievement', '“To', '10PM', 'united', 'states', 'outrageous', 'man,', 'negotiator', 'language', 'lucky', 'politics', 'decision?', 'levels', 'disney', '15%', 'carried', 'interest.', '"In', 'seen', 'divided', 'guess', 'canada', 'sell', 'keystone.', 'south', 'carolina.', 'eyes', '@Newsmax_Media', 'carolina', 'change', 'obsessed', 'it."', 'administration,', 'isn\'t', 'before.', 'shut', 'weak.', 'bowl', 'win,', 'successful', 'MUST', 'tie', 'approved.', 'hundreds', 'steal', 'creates', 'budget.', 'control.', 'shame', 'wasn\'t', 'wind', 'heard', 'bad!', 'smart.', 'owners', 'employees', 'is,', 'notice', 'terrible.', 'israel.', 'union', '@WSJ', 'earlier', 'wanting', 'shoot', 'level', 'allowed', 'obama:', 'world."', 'tonight.', 'paying', '@ApprenticeNBC,', 'primary.', 'claims', 'electric', '@IvankaTrump\'s', 'stamps', '@FaceTheNation', 'lies', '@TrumpNewYork', 'luxury', 'started', 'glad', 'family.', 'supported', 'nasty.', 'remember', 'main', 'mubarak', 'results', 'possible', '@MittRomney\'s', 'hasn\'t', 'allies', 'cancel', 'camp', 'disaster!', 'out!', 'formerly', 'times,', 'critic', 'forced', 'florida.', 'pressure.', 'stock', 'owned', 'german', 'FL', 'housing', 'market,', 'deep', 'senior', 'february', 'tower,', '@MittRomney.', 'bet', 'dumbest', 'thrown', 'sleepy', '@chucktodd', 'prayer', 'breakfast', 'words.', 'you’re', 'airing', '21', 'following', 'victory.', 'irrelevant', '@MittRomney,', 'commercial', 'speak', 'badly', 'talented', 'world,', 'tiny', 'you.', 'admit', 'safety', 'hide', 'truth.', 'bible', 'income', 'is!', '@ericbolling', 'numbers,', 'AM', 'jobless', 'finest', 'hardly', 'trust', 'numbers.', 'staff.', 'many,', 'risk', '@FLGovScott', 'job.', 'failing.', 'sales', '20', 'magazine', '7:00', 'talks', 'solid', 'charges', 'surged', 'catholic', '"I\'m', 'picture', 'no,', 'senate', 'windfarms', '"no', 'whether', 'receive', 'yes,', 'inflation', 'hits', 'secret', 'doing.', 'energy,', 'church', 'lied', 'fastest', 'doesn’t', 'someone', 'it,', 'god', '“Our', '$25', 'slow', 'stupid!', 'talent.', 'houston,', 'goal', 'wins.', '@CelebApprentice', 'cards.', 'sees', 'foolish', 'bringing', '2013', 'congressional', 'right.', 'hosting', 'leader', 'VP', 'reality.', 'xi', 'music', 'early', '#CelebApprentice', 'yesterday,', '500', '@jimmyfallon', 'acting', 'whole', 'shot', 'west', 'bad,', 'good,', 'navy', 'scenes', 'HQ', 'woody', 'others?', 'post', 'world-class', 'property.', 'historic', '"the', 'regulation', 'stopping', 'hiring', 'considering', 'issuing', 'market.', 'good!', 'bush', 'adding', '$6T', 'years.', 'safe!', 'back!', 'wrote', 'criticized', 'forcing', 'secretary', 'address', 'star', 'reviews.', 'words,', '$5', 'euro', 'behalf', 'purchase', 'michigan', 'gone', 'XL', 'broken', 'promises', 'trying', 'experience', 'rev.', 'graham', 'point.', 'eye', 'christians', 'relationship', 'explain', 'headed', '#CNNDebate', 'conservative', 'restore', 'future.', 'pastor', 'christian', 'games', 'played', 'easily', 'country,', 'schultz', 'hell!', 'amazing!', 'tour', 'campaigned', 'pushing', '@WhiteHouse', 'points', 'according', 'able', 'represents', 'boring,', 'academy', '@VanityFair', 'leads', 'proposal', 
#               'strongly', 'hate', 'arabia', '#Oscars', 'attacking', 'survive', 'UK', '"Trump', '=', '7:30', 'fundraising', 'carbon', 'Trump', 'incorrectly', 'stated', 'mitt\'s', 'land', 'gasoline', 'apology', 'insurance', 'illegal', 'healthcare.', 'entrepreneurs', 'night!', 'tuesday,', 'drop', 'worry', 'stronger', 'iraqi', 'missed.', 'pathetic', 'property', 'dopey', 'overrated', 'pundit', 'business,', 'recorded', 'ohio', 'ohio.', 'estimates', 'country!', 'again,', '18', 'delegates.', 'song', 'delegates', 'policy.', 'IRS', '@IsraeliPM', 'promised', 'gitmo', '42', 'presidents', 'county', 'OH', 'crucial', 'democrat', 'california.', 'professional', 'contestants', 'ray', 'kelly', 'commissioner', 'thru', 'acquisition', 'doral', 'class,', 'best.', 'fundraisers', 'personally,', 'NYPD', 'monitoring', 'certain', 'barry', '#TrumpTuesday', 'time,', 'stupidly', 'then,', 'addition', 'review', 'things.', 'hole', 'winner', 'treat', 'views', 'missing', 'much!', 'cadillac', 'championship', 'amazing.', 'NYC\'s', '&amp;', 'Five-Star', 'picks', 'law,', 'terms', 'domestic', 'larger', 'reduce', 'proposed', 'immigrant', 'phone', 'wish', 'religious', 'killers', 'british', 'large', 'barack', 'AM.', 'causing', 'consumer', 'us,', 'are.', 'syria,', 'dumb', 'exposing', 'longest', '38', 'down!', 'investigation', 'justice.', 'perfect', 'star,', 'seriously', 'moron', 'zimmerman', 'nominee', 'saudis', 'child', 'legendary', 'johnson', '@nyjets', 'martin', 'happened.', 'promise', 'company,', 'afghan', 'judges', 'careful!', 'young', 'entrepreneur', 'entering', 'highly', 'beating', 'timing', 'tape', 'ridiculous', 'sound', 'USC', 'dump', 'lies.', 'supreme', 'court', 'decision.', 'filming', 'alex', 'warming', 'dose', 'welcome', 'possibly', 'training', 'election?', 'system.', 'almost', 'winners.', 'players', 'great,', '#TheArtofTheDeal', 'voted', 'health', 'newspaper', '2008', 'necessarily', 'mission', 'percent', '@gallupnews', 'reports', 'vaccinations', 'big.', 'have.', 'desperate', '@MissUniverse', 'yesterday-', '$1T', 'first-ever', 'achievers', 'jose', 'convention', 'center', 'sec.', 'club,', 'charlotte', 'tomorrow.', 'autism', 'competitors', 'aid', 'wisconsin', 'enthusiastic', 'enrollment', 'years,', 'unfairly', 'try', 'advantage', 'general', 'life,', 'estate', 'reform', 'worried', 'especially', 'motivation', 'joined', 'interviews', 'today-', 'account', 'bad.', 'better!', 'hell', 'pennsylvania', 'budget,', 'voting', 'i’ll', '@JudgeJeanine', '#CelebrityApprentice', 'PM.', 'hand', 'success."', 'arrived', 'beach,', 'chris', 'standing', 'miami.', 'payments', 'america!', 'enemies', 'here.', 'independent', 'jenna', 'heavy', 'magazine,', 'failing', 'badly.', '60', 'minutes', 'thomas', 'died.', 'happen', 'beauty', 'heat', 'critics', 'graydon', 'carter', 'spy', 'already.', 'warren', 'walter', 'stance', 'woman', 'doubt', 'keynoting', 'NC', 'school', 'thing.', 'race.', 'onto', 'mitt!', 'fake', 'round', 'sharp', 'ruined', 'november.', 'deals!', 'involved,', 'fronts', 'smart,', 'stick', 'decided', '@FoxBusiness', 'i’ve', 'letters', 'today?', 'matter', 'twitter.', 'wearing', 'attention', 'choose', 'confidence.', '@Macys', 'back,', 'industrial', 'turbines.', 'joint', 'cheat', 'action.', 'investors', 'LA', 'allowing', 'mind', 'practice', 'luck.', 'sign', '100', 'bloomberg', 'mayor', '@nbcsnl', 'yuan', 'tweeting', 'welcoming', 'agent', 'service', 'revealed', 'lack', 'circulation', 'increasing', 'fragrance', 'ABC', 'demos', 'knew', 'disgusting', 'sleazebag', 'film', 'desire', 'awarded', 'ted', 'figure', 'speech,', 'anger', 'immigration', '1st', 'yr', 'lived', 'east', 'everybody', 'views.', 'where\'s', 'opened', 'magnificent', 'hit.', 'manager', '@NBC.', 'dem', 'credibility', 'presidency.', 'scotland,', 'preparing', 'via', 'georgia', 'with.', 'farms', 'lake', 'norman', 'favor', 'loyal', 'enjoying', 'heart', 'is.', 'soon.', 'certificate', 'vast', 'planet', 'runs', 'delivered', 'terror', 'residential', 'gov\'t', 'saying', '@nbc', 'EST!', 'together,', 'turbines', 'threaten', 'birds', '@NBC', 'fixing', 'economy,', 'trial', 'mother', 'turns', 'stole', 'himself.', 'strike', 'design', 'citizens', 'previous', 'official', 'innocent', 'lowering', 'attitude', 'challenge.', 'editorial', 'innovation', 'america"', 'dark', 'knight', 'rises', 'movie', 'orders', 'explains', '@washingtonpost', 'college', 'releasing', 'age', 'negotiations', 'haven\'t', 'joke.', 'fracking', 'bedminster', '2017', 'women\'s', 'jeter', 'fast.', 'labor', 'NEW', '#SweepsTweet', 'target', 'head', 'happen.', 'good.', '#sweepstweet', 'negotiation', 'model', 'happens', 'product', 'fell', 'tool.', 'underestimate', '@johnrich', 'they’re', 'jack', 'feeling', 'instincts', 'listening', 'firing', 'surprise.', 'losers', 'tried', 'competitor.', 'france', 'embrace', 'mess!', 'reputation', 'quote', 'henry', 'far.', 'VOTE', '(&amp;', 'consider', 'writing', 'essentially', 'keynote', 'africa', 'albert', 'tenacious.', 'charles', 'ceremony.', 'polling', 'attacked', 'talent', 'loser!', 'unity', 'gay', 'memory', 'stories', 'handling', 'ugly', 'ruining', 'parts', 'howard', 'stern', 'TIME', 'cover', 'showing', 'jamie', 'ago,', 'break', 'NJ', 'canada,', 'train', 'bank', 'loans', 'citizenship', 'daily', 'hurt', 'ed', 'president,', 'japan', 'korea', 'steve', 'wright', 'philippines', 'aggressive', 'liberty', 'done.', 'it?', 'ObamaCare,', 'ford', 'regulations', 'famous', 'rush', 'supports', 'gaming', 'issue.', 'unemployed', 'CEO', 'watch,', 'NATO', 'summit', 'equity', 'experience,', 'raises', 'franklin', 'violent', 'sitting', 'do.', 'donna', 'careful', 'hired', 'construction', 'leader,', 'funeral', 'closer', 'pretty', 'cable', 'days.', 'win?', 'boardroom', 'ads', 'charity.', 'career.', '@GolfChannel', 'fun.', '@Yankees', 'player', '400', 'also,', 'AAA', 'rating', 'gained', 'smith', 'red', 'mr.', 'economy!', 'song,', '(in', 'African-American', 'registered', 'RNC', 'columbia', 'loss', 'segment', 'throw', 'POTUS.', 'success,', 'be!', 'quality', 'management', 'congrats!', 'ralph', 'waldo', 'emerson', 'exact', 'opposite', 'thanks,', 'homeland', 'city', 'park', 'winston', 'churchill', 'him,', '.@BarackObama', 'begging', 'publicity', 'themselves.', 'club.', 'embarrassed', 'guest', 'fundraiser', 'PAC', 'convention,', '@MissUSA', 'video-', 'added', 'ago', 'celebrated', 'criticizing', 'human', 'rights', 'refugees', 'donations', 'executives', 'commodity', 'choice,', 'aristotle', 'mine', 'unite', 'reverend', 'records', 'me?', 'fact,', 'jobs!', 'agents', 'McCain', 'nomination', 'ironic', 'rule', 'yourself', 'dinner.', '@davidaxelrod', 'write', 'pathetic!', 'difference', 'others', 'down.', 'bridge', 'june', 'hollywood', 'clinton,', 'telling', 'boston', 'yesterday!', 'per', 'falsely', 'horrendous', 'outside', 'fun', 'carly', 'rhode', 'island', 'olivia', 'carry', 'incompetence', 'women.', 'worked', 'location', 'results.', 'pick', 'lowest', 'plan,', 'going.', 'democrats.', 'roger', 'you,', 'help!', 'ran', '.@MittRomney', 'properties', 'broken.', 'serve', 'loser', 'pennsylvania.', 'sun', 'tzu', 'writes', 'hell.', 'amounts', 'constant', 'honor.', 'scheduled', 'miami', 'woman,', 'statements.', 'different', 'closed', 'finally,', '.@CNN', '@MSNBC', 'crazy!', 'teach', 'grant', 'justice', 'leaks', 'situation.', 'S&amp;P', 'builds', 'auto', '@PGATOUR', 'pulling', 'stories.', 'goldman', 'based', 'asia', 'pacific', '"Success', 'final,', 'fatal:', 'courage', 'suing', 'rolls', 'legal', 'U.S.,', 'done!', '@VP', 'nonsense', 'us!', 'positive', 'measure', 'lombardi', 'elected', 'favorability', 'picking', 'sports', 'impress', 'absolute', 'else', 'D.', 'basketball', 'owner', 'applications', '@GOP', 'fred', 'yourself.', 'welch', 'syria', 'offers', 'putin', 'congress!', 'strategy', 'cities', 'airports', 're:', 'w/', 'russians', 'syrian', 'body', 'business.', 'values', 'mexico!', 'administration.', 'ruling', 'result.', 'rape', 'military.', 'brian', 'stood', 'wayne', 'edison', 'staying', 'spin', 'DNC', 'politics!', 'sometimes', 'won.', 'predicted,', 'egypt.', 'increases', 'representatives', 'thursday.', 'development', 'developments', 'test', 'lesson', 'sanders', 'life.', '8%', '40', 'months.', 'attending', 'convention.', 'democratic', 'near', 'usual,', 'wow,', 'society', '@GStephanopoulos', '"not', 'individual', 'ruled', 'constitutional', 'expensive', 'bankrupt', 'chair', 'avoid', 'local', 'roberts', 'loving', 'battle', 'kennedy', 'himself', 'sticking', 'light', 'party!', 'online', 'november!', 'climate', 'command', 'douglas', 'MacArthur', 'apple', 'settle', 'case.', 'par', 'daily.', 'ship', 'diamond', 'con', 'suit', 'superb', 'evangelical', '@TheBrodyFile', 'class.', 'treated', 'fairly.', 'targeted', 'subject', 'say,', 'country?', 'safe.', 'apologize', 'sanctuary', 'F.', 'scott', 'terrible!', 'etc.', 'rid', 'days,', 'wise', 'lie', '@IvankaTrump,', 'sarasota', 'reveals', 'releases', 'likely', 'focusing', 'records.', 'true.', 'cuban', 'richard', '@MLB', 'All-Star', 'game.', 'granted', 'gross', 'sources', 'passion,', 
#               'ultimately', 'or,', '"What', 'matters', 'well,', 'seth', 'co-hosting', 'years?', 'nasty', 'not!', '@AP', 'shouldn\'t', 'ineffective', 'penn', 'ashamed', 'hiding', 'crimes', 'football', 'opponent', 'paper', 'advertising', '18%', 'warmest', 'wishes.', 'sexual', 'pervert', 'huma', 'A-Rod', 'contract', 'correct.', '\'Trump', 'website', '@OMAROSA', 'access', 'space', 'change.', 'interesting!', 'sealed', 'lawyers', 'lawsuits', '@politico', 'year"', 'passport', 'sake', 'poll,', 'TAX', 'fail', 'citizen', 'criminal', 'deal!', 'said,', 'anymore.', 'nelson', 'nothing,', 'die', 'concerning', 'clinton', 'africa.', 'peter', 'breitbart', 'year,', 'wear', 'wounded', 'mass', 'impressed', 'scam', 'built', 'worse.', 'laws', 'suffering', 'luxurious', '$1M', 'NYC.', 'serving', 'guts', 'investments', 'blame', 'on,', 'chavez', 'open.', 'spreading', '"Trump:', 'died', 'handle', 'university.', 'play.', 'bank.', 'to.', 'rough', 'will.', 'corruption', '@whitehouse', '@JoeBiden', 'office,', 'so!', 'jump', 'seen.', 'enthusiasm', 'confident', 'change!', 'role', 'dems', 'skills.', 'apologized', 'illegally', 'act.', 'polls,', 'value', 'filled', 'coverage', 'GO', '#1.', 'anybody', 'member', '@jack_welch', 'all.', 'article:', 'happened', ':', 'demands', 'november', 'senator.', 'suffered', 'losses', 'donated', 'workforce', 'terrific!', 'modern', 'warning', 'credible', 'losers,', 'T.', 'choice.', 'joining', 'us?', 'employ', '@RNC', 'plane', 'ads.', 'hitting', 'ending', 'inaccurate', 'killer', 'scandal', 'worse!', 'dirty', 'renovated', 'election,', 'father', 'william', 'james', 'markets', 'brand', 'working.', '\'Donald', 'count', '22', 'disgraceful', 'son,', 'respected', 'board', 'resign', 'knowingly', 'watch!', 'award.', 'prime', '@nytimes', 'fair.', 'proud!', 'roosevelt', '@foxandfriends.', '@PaulRyanVP', 'base', 'changes', 'ultra', 'machine', 'victory!', 'BIG', 'public.', '‘All', 'star’', 'toward', 'scary.', 'including', 'yesterday’s', 'PGA', 'true,', 'delivering', 'spirit', 'cross', 'lightweight', 'known', 'joke', 'indiana.', 'hopefully,', 'scary', 'architect', 'monster', 'doral.', 'designed', '2016', 'schedule.', 'fronting', '"Mr.', 'huffington', 'AOL', 'president?', 'internet', 'epidemic', '@WWE', 'prior', 'farm', 'harder', 'work,', 'samuel', 'additional', 'plenty', 'economically', 'discussed', 'brother', 'hospital', 'help.', 'virginia', 'governor.', 'feature', 'moves', 'arnold', 'strong!', 'qualified', 'tuesday.', 'friends!', 'cast.', 'tired', 'surprised', 'one,', 'be.', 'helps', 'shirts,', 'them!', 'russian', 'mexico.', '@DNC', 'destination,', '"@realDonaldTrump', 'finance', 'period', '2020', 'course,', 'legacy', 'dishonest', 'SEAL', '2016.', 'accomplished', 'brings', 'documentary', '@NCGOP', 'canadian', 'apologizing', 'birther', 'honoring', '@MELANIATRUMP', '@realDonaldTrump', 'and,', 'quite', '@hardball_chris', 'managing', 'armstrong', 'silent', 'weather', 'fuel', 'pray', 'worry,', 'inside', '@ariannahuff', 'phony', 'follow.', 'mess.', 'etc', 'you-', 'me-', '(with', '@CNNPolitics', 'votes.', 'knock', 'earth', 'lie.', 'endorsing', 'course.', 'together', 'advised', 'shooting', '@CelebApprentice.', 'speech.', 'brothers', 'knockout', 'sends', 'signature', '@Late_Show', 'racist', 'others,', 'jordan', 'polls.', 'assault', 'tougher', 'tennis', 'either', 'comment', 'ball', 'incredible.', 'environment', 'excellence', 'bombed', '13th', 'report.', 'soft', '$\'s', 'african', 'hillary\'s', 'involved', 'ago.', 'stolen', 'rest', 'angry', 'responsibility', 'T.V.', '#TRUMP', 'networks', 'states,', 'possible!', 'changing', 'fill', 'released.', 'hillary', '$10', 'history,', 'virginia.', '@ApprenticeNBC.', 'presidency,', 'sometime', 'taunting', 'REAL', 'brilliant', 'hoax', 'S.', 'patton', '@thehill', 'remembrance', 'media?', 'them?', 'intel', 'blow', 'ambassador', 'fought', 'capable', 'embassies', 'power.', 'besides', 'vets', 
#               'refuse', 'speaks', 'vince', 'started.', 'picked', 'pathetic.', 'monday,', 'bias', 'topped', 'exclusively', 'shirts', 'oh', 'captain', 'tied', 'tweeted', 'handing', 'disaster,', 'kate', 'biased', 'elite', 'abraham', 'lincoln', 'sold', 'london', 'leaked', 'know,', 'dumb!', '13', 'opponents', 'end.', 'turbine', '#Trump', 'families', 'consecutive', 'successful,', '‘The', 'matt', 'D.C.', 'video.', 'einstein', 'woman.', 'failures', 'benghazi', 'attack.', 'univision', 'fun,', 'cup', 'chancellor', 'jerry', 'students', 'demanding', 'nothing.', 'joe,', 'andrew', 'team.', 'emmy', 'politics,', 'influence', '@LibertyU', 'constitution', 'secure', 'impossible', 'again!', 'U', 'replacement', 'considered', 'destruction', 'williams', 'hard.', 'complaints', 'agenda', 'important.', 'O', 'others)', 'column', 'do,', 'ferry', 'arianna', 'associated', 'DJT', 'bomb', 'won!', '@StJude', 'crowd.', 'morons', 'concert', 'arizona', 'faster', 'best!', 'mind.', 'negate', 'patient', 'formidable', 'los', 'palos', '@TrumpGolfLA', 'possible.', 'of,', '36', 'restaurant', '(I', 'credibility!', 'praise', 'saved', 'colorado', 'often', 'block', 'sequester', 'tomorrow\'s', '35', 'audience', 'katie', 'coach', 'nov.', '70', 'watching.', 'place.', '@nfl', '@foxnews', 'obama?', 'arrested', 'stream', 'reporting', 'replace', 'heading', 'friend,', 'wharton', '@DailyCaller', 'greatly', 'twice', 'effective', 'oscar', 'served', 'positive.', 'alone.', '20th', 'luck', 'trail.', 'pearl', 'opposed', 'equal', 'thanks!', 'person.', '#Debate', 'kept', 'momentum', 'sunday’s', '(just', 'salmond', 'flight', 'support!', 'request', '48', 'planned', 'hard!', 'election!', '“In', 'improve', 'obama’s', 'vice', 'windmills', 'this,', 'vladimir', 'A-Rod.', 'hillary.', 'fraud!', 'thinking?', '7.8%', 'throwing', 'tourists', 'atrium', 'traveling', 'nat\'l', 'provide', 'apartment', 'lance', '@yankees', '9th', 'decisions', 'sounds', 'DC.', 'honest', '#VPDebate', 'can.', 'moment', 'ryan', 'fool', 'bird', 'scale', 'minister', 'special!', 'families.', 'leaves', 'capacity', 'drugs', 'reason.', 'everything,', 'steel', 'away.', 'late.', 'trouble', 'either.', 'diet', 'smarter', 'guilty', 'sorry,', '@Macys.', 'boy', 'politician', 'announces', 'donate', 'fails', 'more.', 'gov.', 'gary', 'go!', '.@antbaxter', 'have,', 'opinion,', 'stuff', 'scandals', 'terrorists', '24', 'talk.', '"a', 'immigration.', 'clinton.', '#debate', 'other.', 'broke', 'conflict', 'finances', 'opinion', 'beaten', 'knocked', 'tops', 'necessary', 'girls', 'remove', 'cell', 'add', 'up,', '@extratv', '@Joan_Rivers', 'money,', 'fixed', 'construction.', 'benjamin', 'certainly', 'anymore!', 'president!', 'wall.', 'word', 'bruce', 'don’t', 'blew', 'monday.', 'clean', 'truth!', 'did.', '@BillyGraham', 'carter,', 'past.', 'fort', 'obama!', '@AlexSalmond', 'probably', '“Donald', '“Trump', 'windfarm', 'common', 'card', 'republicans!', 'witch', '@Patriots', 'hampshire!', 'baby', 'felt', 'insane', 'but,', 'dealt', 'refused', 'treatment', 'success!', 'attack,', 'pro', 'see!', 'lying', 'CIA', 'O.K.', 'senate!', 'forward.', 'talent,', 'steven', 'informed', 'night’s', 'tomorrow’s', 'garbage', 'phones', 'stated,', 'virtually', 'bush,', '$5M', 'trending', 'MILLION', 'OF', 'charitable', 'stands', 'waiting', 'side', 'offering', 'shouldn’t', 'six', 'refuses', 'heroes', '@TraceAdkins', 'ovation', 'naming', 'trouble!', 'predict', 'wow!', 'VOTE!', 'silent.', 'room', 'OUT', 'won’t', 'begged', 'team,', 'challenge', 'liked', 'hair', 'extremely', 'politically', '@BillMaher', 'hearing', 'storm', 'nearly', 'properly', 'water', 'more!', 'money!', 'sandy', 'focus,', 'building,', '.@oreillyfactor', '“I', 'date', 'article.', 'media.', 'dummy', '@billmaher', '@NewYorkObserver', 'victims', 'dale', 'carnegie', 'hours', 'happy!', 'supporters,', 'chrysler', 'will!', 'various', 'websites', '800', 'nice,', 'hack', 'overlooking', 'girl', 'there,', 'members', 'city,', 'philly', 'incompetent!', 'vote,', 'ideas.', 'jeff', 'warming?', 'ben', 'prepare', '@mcuban', '.@mcuban', 'moved', 'watching!', '“A', 'direction', 'stops', 'anyway.', 'shark', 'tank', '@StephenBaldwin7', 'you."', 'manhattan', 'kick', 'street,', 'concept', 'supporters.', 'detroit', 'cliff', 'can’t', 'measured', 'VA.', 'workers.', 'electricity', 'yet.', 'lets', 'pleasure', 'today,', 'electoral', 'up!', 'needed', 'process', '@NBCNightlyNews', 'happening', 'together!', 'freezing', 'snowing', '$400', 'in,', 'on!', 'over,', 'reasons', 'u', 'ice', 'agree!', 'patrol', 'heavily', 'awareness', 'aberdeen', 'somebody', 'condolences', 'petraeus', 'losses.', 'pundits', 'tourism', 'reviews', 'calm', 'dishonest!', 'obsolete', 'suits', 'bar', 'gen.', 'angry!', '90', 'dining', 'all,', 'aspiring', 'entrepreneurs,', 'renovations', 'truman', 'country’s', 'appreciated.', 'jupiter,', '@latoyajackson', 'apart', 'iconic', 'york!', 'them.”', 'together.', 'itself', 'door', 'beginning.', 'injured', '(which', 'reached', '72', 'big!', '@HuffingtonPost', 'twain', 'nation.', '@FoxNewsInsider', 'things,', '@lisarinna', 'charge', 'see,', 'THE', 'ART', 'contact', 'conversation', 'yourself."', 'tough!', '...and', 'harry', 'difficulties', 'war.', 'judge', 'broadcast', 'myself', 'HAPPY', '@THEGaryBusey', 'everywhere.', 'glass', 'succeed', 'chance!', 'chance.', 'goals.', 'areas', 'energy!', 'landed', 'goals', 'be.”', 'learn.', '"One', 'you.”', 'restaurant,', 'features', 'settled', 'couldn’t', '@jacknicklaus', 'bit', 'officials', 'race,', 'one!', 'controlled', 'glenfiddich', '“the', 'completed', '@LilJon', 'editor', 'true!', 'embarrassment', 'course”', 'anna', 'wintour', 'smart!', 'on.', 'correct', 'missed!', 'thing!', 'cards', 'scotland’s', 'dropping', 'U.S.!', 'today’s', 'nothing!', 'others.', 'lord', 'sugar', '@Lord_Sugar', 'wouldn’t', '“There', 'facing', 'are!', 'fired.', 'not,', 'ends', 'easy!', '@Morning_Joe', 'slams', 'unit', '46', '“You', 'people,', 'long.', 'banned', 'dope', 'news,', 'WIN', 'PRESIDENTIAL', '@mboyle1', '@BreitbartNews', 'before!', 'suffer', 'setbacks', 'aim', 'high,', '@pennjillette', 'backed', 'dog.', 'enjoy.', 'remarks', 'rigged', 'dress', 'ever!', 'mistake,', 'crowned', '@AP:', 'SAD!', 'hard,', 'complex', 'rio', 'de', 'deals,', 'merry', 'it"', '@dennisrodman', 'dennis', 'game!', 'phil', 'trump”', 'creative', 'hagel', 'it.”', 'family!', 'negotiators', 'history!', 'mitch', 'McConnell', 'eve', 'doctors', 'made.', 'premieres', 'area', 'productive', 'thx', '“What', 'opposes', '“Success', 'breeds', 'alabama,', 'representative', 'knowledge', 'genius', 'that!', 'center.', 'covering', 'becomes', 'passion', 'maher', 'stephen', 'USA.', 'education', 'above', 'folks,', 'investor', 'information.', 'where’s', 'agreement', '“Do', 'sued', 'hated', 'manti', 'disgrace', 'correct!', 'committed', 'pete', 'fame.', 'alabama', '@bretmichaels', 'refusing', 'president’s', 'republicans,', 'confidence', 'approval.', 'responsible.', 'something.', 'me!', 'inaugural', 'premiums', 'enjoy!', 'wrong,', '@BreitbartNews:', 'IN', '.@THEGaryBusey', 'legislation', '“Always', 'treats', 'wallace', 'referred', 'referring', 'farmers', '(but', 'shocked', 'what’s', 'names', '.@AlexSalmond', 'evidence', 'run.', 'poorly', 'prisoners', 'vietnam', 'there’s', '“Be', 'puts', 'yourself,', 'blip', 'from.', 'lights', 'seven', 'lawsuit', 'party,', 'acres', 'doing!', 'checks.', 'C.', 'yourself.”', 'mary', 'MacLeod', 'entrepreneurs:', '@DoralResort', 'WGC', '@cadillacchamp', 'hurting', 'happening!', 'everyone,', 'persuasion', 'solution,', 'momentum.', 'momentum,', 'nowhere.', 'morning!', 'personable,', 'personally.', 'quickly,', 'you’ll', 'commercials', '@AmSpec', 'reasonable', 'blown', 'lies,', 'pols', 'ashley', 'judd', 'effort.', 'fear', '.@KarlRove', 'harvey', 'blasts', 'haters', 'sets', 'horrific', 'pretend', 'simple,', 'opportunities.', 'email', 'largely', 'frack', 'dr.', 'carson', 'marco', 'rubio', 'speech!', '@marklevinshow', 'arena', 'yourself:', 'standard', 'identify', '@TrumpToronto', 'airlines', 'AS', '“Trump:', 'immigrants', 'amnesty', 'america,', 'rocket', 'carolina,', '@AFPhq', 'expected.', 'stamina', 'direction.', 'tread', 'director', 'ARE', 'optimism', 'acknowledging', 'merely', 'P.M.', 'minutes!', 'cufflinks', 'oscars', 'barbara', 'icahn,', 'far,', '"DONALD', 'joke!', 'inducted', 'late!', '(a', 'event,', 'job,', 'ties,', 'madison', 'garden', 'beats', '@CadillacChamp', 'happen!', 'A.M.', 'ENJOY!', '$200M', 'away!', 'losers.', 'korea,', '(not', 'doing,', 'boardroom.', 'venezuela', 'yourself!', 'korea.', 'so,', 'importantly,', 'aberdeenshire', '@CPACnews', 'AT', 'television.', 'wow', 'tiger', 'big,', 'DONALD', 'resolve', 'protection', 'NOTHING!', 'tee', 'shortly.', 'teams', 'task', 'grateful', 'definition', 'int.', '@TigerWoods', '$250M', 'elizabeth', 'potomac', 'ads,', 'years!', 'selection', 'remains', 'accurate', 'BACK', 'MEDIA', 'ON', '“TRUMP', 'BUILD', 'GREAT', 'WE', 'BE', 'SO', 'MUCH', 'crowd,', 'end!', 'emergency', 'unfair', 'CBS', 'monstrosity', 'courts', 'championships', 'IS', 'GET', 
#               '(the', 'india', 'nation’s', 'returned', 'episodes', 'expanded', 'do!', 'NFL', '@WrestleMania', '@msnbc', 'haters,', 'rules.', 'can,', 'amendment.', 'stupidity', 'coldest', 'suggest', '@TrumpTowerNY', '@TrumpTowerNY.', 'wasn’t', 'owe', 'headline', 'approval', 'extraordinary', '.@alexsalmond', '@pressjournal', '@NYMag', '@autismspeaks', 'fortune', 'all!', '“Never', 'story.', 'hates', 'representing', 'immigration,', 'illegals', 'elections', 'N.Y.', 'bills', 'korean', 'negotiation:', 'anonymous', 'mrs.', '#WWEHOF', 'wine', 'fan!', 'failed,', 'finished', '.@foxandfriends', 'margaret', '@trumpwinery', '#TrumpVlog', 'dallas', 'NEVER', '#TBT', 'indiana', 'amenities', 'close!', 'kerry', 'JUST', 'GREAT!', 'dumber', 'down,', 'lawyer', 'responders', 'sexting', 'panoramic', 'views,', 'spa', 'destination', 'sunday.', 'media,', 'NO', 'pleased', 'brett', 'minutes).', 'thug', 'bombing', 'register', 'jonathan', 'medical', 'sen.', '.@VattenfallGroup', 'prison', 'upon', 'addressing', 'will,', 'price.', '7.00', 'size', '.@DannyZuker', '@DannyZuker', 'trump!', 'lie!', 'tens', 'snow', '—', 'ZERO', '“Don’t', 'babe', 'ruth', 'jodi', 'jury', 'MAKE', 'wife,', 'melania,', 'puerto', 'rico', 'danger', 'clue!', 'dubai.', 'hilton', 'assaults', 'this!', 'lee', 'generals', 'dope!', 'buffett', '#FundAnything', 'learning', 'ever,', 'THAT', 'HAS', 'FOR', 'BY', 'talents', 'control,', 'sebelius', 'on?', 'MANY', 'going!', 'efforts', 'allows', 'rivers', 'oklahoma', 'reporting.', 'vicious', 'facts', 'bless', 'pouring', 'nights', 'losers!', 'bosses', 'infrastructure', 'AMERICA', 'AGAIN!', 'unable', 'invest', 'horribly', 'windy', '@TrumpChicago', 'soars', 'combination', '2016,', 'world!', 'bay,', '@TrumpPanama', 'architectural', 'open!', 'IMMIGRATION', 'crime,', '@fundanything', 'county,', 'collection,', '@Macys,', 'focused.', 'report:', 'NSA', 'mexico', 'person!', '@WashTimes', 'snowden', 'forgotten', 'nice!', 'WILL', 'OUR', 'U.S.A.', '(by', 'apology!', 'moscow', 'name.', 'deleted', 'fabricated', 'careful,', 'art.', 'present.', 'hunt', '@theFAMiLYLEADER', 'senators', 'breaking', 'worker', '@TrumpDoral’s', 'groups', 'amnesty.', 'unnecessary', '@TrumpTO', 'minutes.', 'court!', '19', 'edward', '@TrumpNationalNY', 'mandela', 'father,', 'ego', 'L.A.', 'frisk', 'yorkers', 'trade,', 'TOLD', 'YOU.', 'results!', 'dwight', 'verdes', 'peninsula', '@mboyle1:', 'spitzer', 'inspired', '@NRO', 'crash', 'eliot', 'A.G.', 'wacko', 'lord:', 'traitor', 'crime!', 'core', 'watch-', 'russia.', 'accomplish', 'concentrated', 'vital', '360', 'NOW!', 'masterpiece', 'screen', 'RECORD', '#TrumpVine', '@bobvanderplaats’', 'crowd!', 'bobby', '@TrumpDoral', 'punta', 'del', 'water.', 'surface.', 'tip:', 'doing."', 'appeal', 'voters.', '@Macys.com', 'hotel,', 'leverage.', 'leadership!', 'unbelievable', 'shutdown', '@GolfMagazine', 'style', 'iowa.', '"Best', 'jason', 'NEWS', 'endeavor.', '#TrumpAdvice', 'tool', 'STOP', 'stuart', 'was,', 'we’re', '#FlashbackFriday', 'attorney', 'schneiderman', '98%', 'file', 'contributions', 'moreland', 'AG', 'NY.', 'files', '.@FoxNews', 'fly', 'syria.', 'FIRST', '@businessinsider', 'fiasco', 'coalition', 'pay.', 'knowing', 'fools', 'services', 'AGAIN,', 'VERY', 'DO', 'IF', 'BAD', 'FROM', '(who', '"leaders"', 'states!', 'assad', 'bottom', 'minds', 'THEY', 'IT!', 'exciting!', 'better,', 'prestigious', 'COUNTRY', 'russia,', 'meeting.', '@EricTrump!', 'anyway!', '@FoxNewsSunday', 'thing,', 'cruz', '#MakeDCListen', '$50', 'skills', 'city’s', 'disastrous', '@Trump_Charlotte', 'youth', 'repubs', 'primaries', 'affordable', 'duty', '$’s', 'delay', 'americans!', 'sam', '@AGSchneiderman', 'int’l', 'holes', '@nypost', 'nicklaus', 'bear', 'NOTHING', 'website.', 'TOTAL', 'WORLD', 'dubai', '$1B', 'more,', '7.00.', 'vet', 'IT', 'EVER', 'rumor', 'ObamaCare!', 'leaders,', 'thanks.', 'victorious.', 'imagination', 'PEOPLE!', 'happen,', 'ONLY', 'late,', 'PEOPLE', 'rooms', 'WIN!', '@MariaBartiromo', '@FoxBusiness.', 'ALWAYS', 'racial', 'exposed', 'filibuster', 'made!', 'FAST!', 'texas,', 'GLOBAL', 'TRUMP:', 'example.', '@TrumpChicago’s', 'terrible,', 'don\'t,', 'about!', 'skyline', 'unfair!', 'crooked', 'YOU!', 'minnesota', 'temperatures', 'tennessee', 'florida,', 'b/c', 'talent!', 'WARMING', 'CLIMATE', 'CHANGE', 'BEST', 'horror', 'suites', 'walker', 'DON\'T', 'helicopter', '“No', 'lost!', 'jan.', 'level.', '@BretBaier', '#Trump2016', 'goals,', 'realized', 'clear.', 'appreciated!', '@UnionLeader', 'villa', 'landmark', 'option', 'louisiana,', '@TrumpCollection', 'SAFE', 'reception', 'dems.', 'ireland', 'doonbeg', 'resolution', 'eric,', 'southern', 'discredited', 'hampshire,', 'buffalo', '@BuzzFeed', 'crowds', '@TODAYshow', 
#               'high!', '@krauthammer', 'R.', 'rand', 'kim', 'strange', 'week!', 'actually,', 'baltimore', 'alaska', 'THANK', 'palmer', 'baton', 'rouge,', 'louisiana', 'house!', 'boeing', 'putin.', 'mostly', 'doctor,', 'rarely', '@CNN.', '@Citizens_United', 'jeb', 'granite', '‘I', 'BORDER', 'HAVE', 'convinced', 'pure', 'disrespect', 'counts.', '@buffalobills', 'easter', 'nat’l', 'tragic', 'peaceful', 'rep.', 'NYC,', 'peace', 'incorrect', 'turnberry', 'left.', 'TWO', 'SHOULD', 'disgrace!', 'wild.', 'dead!', 'wages', 'guestrooms', 'cleveland', 'AGAIN', 'montana', 'ireland,', 'criminals', 'persistence', 'VA', 'vets.', 'immediate', 'honor!', '.@TrumpChicago’s', '@TrumpScotland', 'run,', 'bergdahl', 'PRESIDENT', 'mississippi', 'provides', 'marine', 'tahmooressi.', '@Trump_Ireland', 'tahmooressi', '.@MattGinellaGC', 'ISIS', 'sits', 'array', '@leezeldin', '@ForbesInspector', '@NYDailyNews', 'dishonest.', 'thugs', '@TrumpTurnberry', 'senate,', 'stubborn', 'status', 'border.', 'border,', '#FreeOurMarine', '1st.', 'ago!', '|', 'flights', 'ebola', 'KEEP', 'infected', 'roads,', 'eugene', 'WALL!', '2015', 'ISIS.', 'JOBS!', 'virginia,', 'ICYMI,', 'iowa,', 'borders,', 'trumps', 'children,', '@meetthepress', 'record!', 'lift', 'kentucky', 'senate.', 'nunn', 'ebola.', 'forget.', 'ebola,', 'ISIS,', '@DMRegister', '@JenniferJJacobs:', 'wall!', 'etc.,', 'U.', 'joke,', 'lobbyist', 'ban', 'terrorism.', 'A.C.', 'deductibles', 'ICE', '@IvankaTrump!', 'hampshire.', 'tuesday!', 'veteran', '@greta', 'state!', 'gruber', 'desperately', 'congress,', 'border!', 'viciously', '@SixteenChicago', 'sessions', 'replace!', 'venue', 'ferguson', 'riots', '9:00', 'bedminster,', 'oval', 'des', '7:00.', 'fraudulent', 'moines', '“President', '8:00.', 'HRC', '8:00', 'paris', 'myrtle', 'nevada', 'happens!', 'tomorrow,', 'iowa-', 'romney,', '@NBCNews', '51', 'turnberry,', 'MSM', '#FITN', '#MakeAmericaGreatAgain', '@CBSNews', 'NATIONAL', 'WALL', 'video:', 'typical', 'negotiated', 'crowds,', '6:00', 'candidates.', '.@megynkelly', 'media!', 'Trans-Pacific', 'partnership', 'TPP', 'crowds.', 'STATE', 'officers', 'lobbyists', 'U.S.A.!', 'kasich', '@SRQRepublicans', 'clown', 'iowa!', 'supporters!', '#NCGOPcon', 'phyllis', 'poll:', 'surges', 'streaming', '#Periscope:', 'analysis', 'deepest', 'goofy', 'FAR', 'hillary?', 'daughter,', '#MakeAmericaGreatAgain!', '"Trump"', 'security!', 'jeb\'s', 'phoenix,', 'phoenix', 'plant', 'apologize!', '@HillaryClinton', 'Thank', '@andersoncooper', 'anti-Trump', '10:00.', 'LOVE', 'hillary’s', 'emails.', '#GOPDebate', '@oreillyfactor.', 'megyn', 'fiorina', 'shop', 'dow', 'evening!', 'reuters', 'evangelicals', 'france.', '10:00', 'POLL', 'classified', 'clintons', 'texas.', 'self-funding', 'hillary,', 'poll-', 'drudge', 'landing', 'california!', 'afternoon.', '#asktrump', '7pm', 'CARSON', 'CRUZ', 'rubio,', '@60Minutes', 'congratulations!', 'poll!', '@donlemon', 'tennessee,', 'melania.', 'wacky', 'nevada!', '11:00', 'georgia!', '@SeanHannity', 'inform', '#DemDebate', '.@HillaryClinton', 'germany', '.@JebBush', '@OANN', 'purposely', '#GOPDebate.', 'nevada.', 'CRIPPLED', 'polls!', 'kasich,', 'on-line', '10pmE', 'rubio.', 'carolina!', '#IACaucus', 'biased.', 'RADICAL', 'ISLAMIC', 'shortly', '#ISIS', 'tennessee!', 'ohio,', 'terrorism', '#ICYMI:', 'lindsey', 'enforcement', '#LESM', 'FBI', '@nytimes,', 'MOVEMENT!', 'one-sided', 'mainstream', 'rally.', '@NYTimes', '@Netanyahu', 'U.K.', 'podesta', 'M', 'departing', 'cedar', '@JebBush', 'rapids,', 'CLINTON', 'michigan.', 'michigan!', 'cruz.', 'wall,', 'hillary!', 'e-mail', 'BIRTHDAY', 'candidates,', '10,000', 'endorse', 'rallies', '@jdickerson', 'uranium', '#VoteTrump', 'YOU,', 'endorsement.', 'rally!', 'Video:', 'florida!', 'cruz,', 'Poll', 'judgement', '#CaucusForTrump', 'Tickets:', '#VoteTrumpNH', '#NHPrimary', 'S.C.', 'court.', '#VoteTrumpSC', 'R', 'AMERICA!', 'bernie.', 'texas!', 'tickets:', 'florida-', 'virginia!', '#SuperTuesday', 'ohio!', 'corker', 'kansas', 'nancy', 'disgraceful!', 'MOVEMENT', 'NAFTA', 'NAFTA,', '#TrumpTrain', 'lyin\'', 'missouri', 'arizona!', 'arizona.', 'wisconsin!', '#WIPrimary', 'wisconsin.', 'pennsylvania!', '#MAGA', '#NYPrimary', 'indiana!', '#INPrimary', 'pence', 'native', 'warren,', 'heritage', 'come!', 'FIRST!', 'W.H.', '7pm!', 'clinton\'s', 'unfit', '#AmericaFirst', '33,000', 'senators,', 'V.P.', 'tampa,', '#CrookedHillary', '@LouDobbs', '#ImWithYou', 'H', 'JOBS,', '#TrumpPence16', '#RNCinCLE', 'kaine', '@Mike_Pence', 'exist.', '7pm.', '#CrookedHillary.', '#AmericaFirst.', '#MAGA!', 'pensacola,', '#DebateNight', '#Debates2016', 'israel,', '#BigLeagueTruth', 'server', '#Debates', '#ICYMI-', 'comey', '#DrainTheSwamp', '#DrainTheSwamp!', '#ObamacareFail', '#MAGA🇺🇸', 'cabinet', 'MAKING', 'FAKE', 'NEWS!', 'rex', 'briefing', 'schumer', 'WITCH', 'HUNT!', '#FakeNews', 'luther', '#AmericaFirst🇺🇸', 'tariffs', 'abe', 'dems,', '@WhiteHouse!', '@TuckerCarlson', '#USA🇺🇸', '2018', '#WeeklyAddress🇺🇸', 'collusion', '@FLOTUS', 'hunt!', 'GOD', 'MS-13', 'moon', '#G20Summit', 'HealthCare', 'alabama.', 'endorsement!', 'jong', 'un', 'BLESS', 'FEMA', 'DACA.', 'anthem.', 'disrespecting', 'mueller', 'shortly!', 'VOTE,', 'HARASSMENT!']


word_to_index = dict((c, i) for i, c in enumerate(words_list))
index_to_word = dict((i, c) for i, c in enumerate(words_list))
print('there are', len(words_list), 'words')
print(words_list)
words_set = set(words_list)
# ignore all invalid words in tweets
print("BEFORE filtering, there were", len(tweets), "tweets")
new_tweets = []
for tweet in tweets:
    filtered_tweet = filter_words(tweet, words_set)
    if len(filtered_tweet) > 0.8*len(tweet):
        new_tweets.append(filtered_tweet)
tweets = new_tweets

# filter short tweets
tweets = [tweet for tweet in tweets if len(tweet) > MIN_TWEET_LENGTH]

print("AFTER filtering, there are", len(tweets), "tweets")

f = open('/content/trump_words.txt', 'w')
f.write('[' + ', '.join(["'" + w.replace("'", "\\'") + "'" for w in words_list]) + ']')
f.close()


def main():
    pass    # do nothing (may comment out if we want to test something)

if __name__ == '__main__':
    main()


In [ ]:
# filter text file of tweets for ones that are valid inputs for the first 5 words
tweets_list = get_tweets_list('/content/processed_trump_test_data')
def first_5_words_in_dict(text):
    words = text.split(' ')
    if len(words) < 6:
        return False
    for i in range(5):
        if words[i] not in words_set:
            return False
    return True
tweets_list = [t for t in tweets_list if first_5_words_in_dict(t)]
f = open('/content/out', 'w')
for t in tweets_list:
    f.write(t + '\n')
f.close()


Train Model

In [ ]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional, BatchNormalization, Activation
from keras.layers import Dropout
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.utils.data_utils import get_file
import random
import io
from google.colab import files
!pip3 install truecase
import truecase

INPUT_LENGTH = 5  # based on INPUT_LENGTH characters, our model generates the next character
GENERATED_TWEET_LENGTH = 20 # words


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def get_truecase(sentence):
    old_words = sentence.split(' ')
    new_words = truecase.get_true_case(sentence).split(' ')
    print('raw truecase:', ' '.join(new_words))
    for old_word in old_words:
        if not is_normal_capitalization(old_word):
            var1 = old_word.lower()
            var2 = var1[0].upper() + var1[1:]
            if var1 in new_words:
                new_words[new_words.index(var1)] = old_word
            elif var2 in new_words:
                new_words[new_words.index(var2)] = old_word
    return ' '.join(new_words)
    # return ' '.join([new_words[i] if is_normal_capitalization(old_words[i]) else old_words[i] for i in range(len(old_words))])

def on_epoch_end(epoch, _, data, model):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    for _ in range(2):     # use 10 different tweets as samples
        tweet = np.random.choice(data) # select random tweet
        start_index = 0

        for diversity in [0.2, 0.4, 0.6, 1.0]:
        # for diversity in [0.1, 0.2, 0.3, 0.4]:
        # for diversity in [0.3, 0.4, 0.5]:
            print('----- diversity:', diversity)

            generated = ''
            sentence = tweet.split(' ')[start_index: start_index + INPUT_LENGTH]
            generated += ' '.join(sentence)
            print('----- Generating with seed: "' + ' '.join(sentence) + '"')
            # sys.stdout.write(generated)

            for i in range(GENERATED_TWEET_LENGTH):
                x_pred = np.zeros((1, INPUT_LENGTH, len(words_list)))
                for t, word in enumerate(sentence):
                    x_pred[0, t, word_to_index[word]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_word = index_to_word[next_index]
                generated += ' ' + next_word
                sentence = sentence[1:] + [next_word]

                # sys.stdout.write(next_word)
                # sys.stdout.flush()
            print(generated)
            print('with truecase:')
            # preserve
            print(get_truecase(generated))
            print()
    # save and download the model
    model.save('/content/model')
    !zip -r /content/model.zip /content/model
    files.download('/content/model.zip')


def train_from_data(data, train_limit=None):
    # convert the raw tweets list to input and output
    # input is equal to INPUT_LENGTH characters, output is a single character
    if train_limit:
        data = data[:train_limit]
    sentences = []
    next_words = []
    for tweet in data:
        tweet_words = tweet.split(' ')
        for i in range(0, len(tweet_words) - INPUT_LENGTH):
            sentences.append(tweet_words[i: i + INPUT_LENGTH])
            next_words.append(tweet_words[i + INPUT_LENGTH])
    print('# training samples:', len(sentences))
    # for i in range(10):
    #     print(sentences[i],'->',next_words[i])

    # vectorize the data
    print('Vectorization...')
    x = np.zeros((len(sentences), INPUT_LENGTH, len(words_list)), dtype=np.bool)
    y = np.zeros((len(sentences), len(words_list)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, word in enumerate(sentence):
            x[i, t, word_to_index[word]] = 1
        y[i, word_to_index[next_words[i]]] = 1

    # build the model
    print('Build model...')
    model = Sequential()
    model.add(LSTM(128, input_shape=(INPUT_LENGTH, len(words_list))))
    # model.add(LSTM(len(VALID_CHARS) * 7, input_shape=(INPUT_LENGTH, len(VALID_CHARS))))
    
    model.add(BatchNormalization())
    model.add(Activation('selu'))

    model.add(Dense(128))
    model.add(Activation('selu'))

    # model.add(Dense(len(VALID_CHARS)*4))
    # model.add(BatchNormalization())
    # model.add(Activation('selu'))

    # model.add(Bidirectional(LSTM(128), input_shape=(INPUT_LENGTH, len(VALID_CHARS))))
    model.add(Dense(len(words_list), activation='softmax'))

    # optimizer = RMSprop(lr=0.01)
    optimizer = Adam()
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    epochs = 10
    
    print_callback = LambdaCallback(on_epoch_end=lambda a, b: on_epoch_end(a, b, data, model))

    # train the model
    model.fit(x, y,
            epochs=epochs,
            callbacks=[print_callback]
            )

    # save and download the model
    model.save('/content/model')
    !zip -r /content/model.zip /content/model
    files.download('/content/model.zip')

def main():
    print("number of tweets:", len(tweets))
    train_from_data(tweets)


if __name__ == '__main__':
    main()

Code to generate tweets after model is trained

In [ ]:
from tensorflow import keras


NUM_TWEETS_TO_GENERATE = 100
TEMPERATURE = 0.7

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# --- LOAD THE MODEL --- #
# !unzip /content/model.zip
!unzip -o /content/model.zip
model = keras.models.load_model('/content/content/model')

GENERATED_TWEET_LENGTH = 30
INPUT_LENGTH = 5

f = open('/content/model-output.txt', 'w')
f2 = open('/content/out', 'r')
lines = f2.read().split('\n')

# for i in range(NUM_TWEETS_TO_GENERATE):
for tweet in lines:
    # tweet = np.random.choice(tweets) # select random tweet
    start_index = 0
    generated = ''
    sentence = tweet.split(' ')[start_index: start_index + INPUT_LENGTH]
    generated += ' '.join(sentence)
    print('----- Generating with seed: "' + ' '.join(sentence) + '"')

    for i in range(GENERATED_TWEET_LENGTH):
        x_pred = np.zeros((1, INPUT_LENGTH, len(words_list)))
        for t, word in enumerate(sentence):
            x_pred[0, t, word_to_index[word]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, TEMPERATURE)
        next_word = index_to_word[next_index]
        generated += ' ' + next_word
        sentence = sentence[1:] + [next_word]

    print(generated)
    f.write(generated + '\n')
    print()
f.close()
